## Imports

In [2]:
import os
import cv2
import pickle
import imageio
import numpy as np
import pandas as pd
from datetime import datetime
import matplotlib.pyplot as plt
import matplotlib as mpl
mpl.use('Agg')
from matplotlib.animation import FFMpegWriter
from tqdm import tqdm
from pprint import pprint
import sys
sys.path.append('./')
from utils import normalize_image, get_sequences

In [3]:
from enum import Enum
from collections import namedtuple

class ModelType(Enum):
    CLASSIFICATION = 0
    SEGMENTATION = 1

classlabels_viz_colors = ['black', 'green', 'yellow', 'blue', 'red', 'magenta', 'cyan',
                          'lightseagreen', 'brown', 'magenta', 'olive', 'wheat', 'white', 'black']
classlabels_viz_bounds = [-1, 0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 100]

classlabels_viz_cmap = mpl.colors.ListedColormap(classlabels_viz_colors)
classlabels_viz_norm = mpl.colors.BoundaryNorm(classlabels_viz_bounds, classlabels_viz_cmap.N)

confidence_heatmap_viz_colors = ['black', 'blue', 'red', 'orange', 'yellow', 'lightgreen', 'lightseagreen']
confidence_heatmap_viz_bounds = [-1, 0,0.5,0.6,0.7,0.8,0.9,1]
confidence_heatmap_viz_cmap = mpl.colors.ListedColormap(confidence_heatmap_viz_colors)
confidence_heatmap_viz_norm = mpl.colors.BoundaryNorm(confidence_heatmap_viz_bounds, confidence_heatmap_viz_cmap.N)


LabelColor = namedtuple('LabelColor', ['name', 'id', 'trainid', 'color', 'category'])

LABEL_COLORS = [
    LabelColor('class1', 1, 0, (128, 0, 128), 'driveableterrain'),
    LabelColor('class2', 2, 1, (255, 0, 0), 'non-driveableterrain'),
    LabelColor('class3', 3, 2, (0, 0, 255), 'sky'),
    LabelColor('class4', 4, 3, (0, 255, 0), 'trees'),
    LabelColor('class5', 5, 4, (255, 0, 255), 'implement'),
    LabelColor('class6', 6, 5, (255, 255, 0), 'basket markers')
]

LABEL_COLORS_4CLASS = LABEL_COLORS[0:4]
LABEL_COLORS_5CLASS = LABEL_COLORS[0:5]
LABEL_COLORS_6CLASS = LABEL_COLORS[0:6]
LABEL_COLORS_SKY_DET = [LABEL_COLORS[0], LABEL_COLORS[2]]

LABEL_COLORS_IMPL = [
    LabelColor('class1', 1, 1, (128, 0, 128), 'implement'),
    LabelColor('class2', 2, 2, (255, 0, 0), 'sweep'),
    LabelColor('class3', 3, 3, (0, 0, 255), 'harrow_tine'),
    LabelColor('class4', 4, 4, (0, 255, 0), 'basket'),
    LabelColor('class5', 5, 5, (0, 255, 0), 'basket_marker'),
    LabelColor('class6', 0, 255, (0, 0, 0), 'ignore')
]

LABEL_COLORS_IMPL_REDUCED = [
    LabelColor('class0', 1, 0, (0, 0, 0), 'background'),
    LabelColor('class1', 2, 1, (0, 255, 0), 'implement'),
    LabelColor('class2', 3, 2, (255, 0, 0), 'sweep'),
    LabelColor('class3', 4, 3, (0, 255, 0), 'basket_marker'),
    LabelColor('class6', 0, 255, (0, 0, 0), 'ignore')
]

PLUG_LABEL_MAP ={0: 'no-plug', 1: 'plug'}
IMPL_SEGMENT_LABEL_MAP = {0: 'background', 1: 'implement', 2: 'sweep', 3:'basket_marker'}

In [4]:
def create_mpl_viz_outputs(output_path,
                           image,
                           prediction_labels,
                           confidences,
                           depth_img, 
                           image_title='Image', 
                           pred_title='Prediction', 
                           conf_title='Confidence', 
                           depth_title='Depth',
                           bbox_coords=[]):
    """
    Utility function to plot results based on order of input provided.

    Axes index can have different values based on input provided.

    For example: If image, prediction and groundtruth_label is provided, A three axes plot will be generated with
    image(ax1), ground_truth(ax2), prediction(ax3).

    Order of plot if all the inputs are provided will be in same order as arguments listed above.
    """
    axis_index = list(range(len(list(filter(lambda x: x is not None, [image,
                                                                      prediction_labels, confidences,
                                                                      depth_img])))))
    axis_curr_index = 0
    fig, axes = mpl.pyplot.subplots(1, len(axis_index), figsize=((60, 30)))
    if bbox_coords:
#         xmin, ymin, xmax, ymax = bbox_coords
        xmin, xmax, ymin, ymax = bbox_coords
        if (xmin < 0) or (ymin < 0):
            raise ValueError(f'Either {xmin} or {ymin} are negative')
        else:
            rect = mpl.patches.Rectangle((ymin, xmin), (ymax - ymin), (xmax - xmin), linewidth=3, edgecolor='k',
                                     facecolor='none')
    else:
        rect = None

    if axis_curr_index < len(axes):
        # Grab only RGB channels from image, otherwise depth with distort the image when it is displayed
        axes[axis_curr_index].imshow(image)
        axes[axis_curr_index].set_title(image_title, fontsize=30)
        axes[axis_curr_index].axis('off')
        axis_curr_index += 1

    if axis_curr_index < len(axes):
        axes[axis_curr_index].imshow(depth_img, cmap='turbo')
        axes[axis_curr_index].set_title(depth_title, fontsize=30)
        axes[axis_curr_index].axis('off')
        axis_curr_index += 1

    if axis_curr_index < len(axes):
        axes[axis_curr_index].imshow(prediction_labels, classlabels_viz_cmap, classlabels_viz_norm, interpolation='nearest')
        if rect is not None:
            rect1 = mpl.patches.Rectangle((ymin, xmin), (ymax - ymin), (xmax - xmin), linewidth=3, edgecolor='k',facecolor='none')
            axes[axis_curr_index].add_patch(rect1)
        axes[axis_curr_index].set_title(pred_title, fontsize=30)
        axes[axis_curr_index].axis('off')
        axis_curr_index += 1

    if axis_curr_index < len(axes):
        c = np.max(confidences, axis=2)
        axes[axis_curr_index].imshow(c, confidence_heatmap_viz_cmap, confidence_heatmap_viz_norm, interpolation='nearest')
#         if rect is not None:
#             rect2 = mpl.patches.Rectangle((ymin, xmin), (ymax - ymin), (xmax - xmin), linewidth=3, edgecolor='k',
#                                           facecolor='none')
#             axes[axis_curr_index].add_patch(rect2)
        axes[axis_curr_index].set_title(conf_title, fontsize=30)
        axes[axis_curr_index].axis('off')
        axis_curr_index += 1

    mpl.pyplot.savefig(output_path, pad_inches=0, bbox_inches='tight', dpi=150)
    mpl.pyplot.close('all')

def read_image(image_path):
    image = (np.load(image_path) * 255).astype(np.uint8)
    image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)
    return image

def read_saved_frame(pred_dir, image_id):
    states_to_save = ['', 'false_positive', 'false_negative', 'large_object_false_negative', 'true_positive', 'true_negative']
    frame = None
    for state in states_to_save:
        if os.path.isfile(os.path.join(pred_dir, state, image_id+'.png')):
            frame = cv2.imread(os.path.join(pred_dir, state, image_id+'.png'))
            break
        if os.path.isfile(os.path.join(pred_dir, state, image_id+'.jpg')):
            frame = cv2.imread(os.path.join(pred_dir, state, image_id+'.jpg'))
            break
    return frame

def read_images(pred_dir, _id):
    if not os.path.isfile(os.path.join(pred_dir, _id+'_image.npy')):
        return None, None, None, None
    image = np.load(os.path.join(pred_dir, _id+'_image.npy'))
    
    # 100m capped depth
    depth = np.load(os.path.join(pred_dir, _id+'_depth.npy'))
#     # raw depth
#     raw_depth_dir = '/raum_raid/li.yu/data/Jupiter_rock_demo_2021/Jupiter_rock_demo_loamy06_Oct20_2021/model_processed_v4.1_sky_2e-3_lr_1e-3_color_aug_full_model_LR_consistency_regularization_0.2_epoch_23/images/'
#     stereo_data = np.load(os.path.join(raw_depth_dir, _id, 'stereo_output.npz'))
#     depth = stereo_data['point_cloud'][:,:,-1]
#     depth = normalize_and_clip_depth(depth, 200)
    
    pred_label = np.load(os.path.join(pred_dir, _id+'_pred_label.npy'))
    confidence = np.load(os.path.join(pred_dir, _id+'_confidence.npy'))
    return image, depth, pred_label, confidence

# def create_frame(pred_dir, pred_merged_dir, _id, recreate=False):
#     canvas_path = os.path.join(pred_merged_dir, _id+'.png')
#     if recreate:
#         image, depth, pred_label, confidence = read_images(pred_dir, _id)
#         if image is None:
#             return None
#         create_mpl_viz_outputs(canvas_path, image, pred_label, confidence, depth)
#     frame = cv2.imread(canvas_path)
#     return frame

def get_bbox_coords(i=-1, bbox_range_list=[], bbox_coord_list=[]):
    for bi in range(len(bbox_range_list)):
        bbox_range = bbox_range_list[bi]
        if bbox_range[0] <= i <= bbox_range[1]:
            return bbox_coord_list[bi]
    return []

def process_frame(pred_dir, pred_merged_dir, _id, recreate=False, bbox_coords=[]):
    image, depth, pred_label, confidence = None, None, None, None
    l = 0.0
    avg_pixel = 0.0
    bbox_conf = None
    if recreate:
        image, depth, pred_label, confidence = read_images(pred_dir, _id)
        if image is None:
            return image, depth, pred_label, confidence, l, avg_pixel, bbox_conf
        # calculate brightness
        hlsImg = cv2.cvtColor(image, cv2.COLOR_RGB2HLS)
        l = np.average(hlsImg[:,:,1])
        image_title = 'Image (brightness: {:.4f})'.format(l)
        # calculate average pixel value at bbox area
        if bbox_coords:
            ymin, ymax, xmin, xmax = bbox_coords
            bbox_pred = pred_label[ymin:ymax+1, xmin:xmax+1]
            bbox_conf = confidence[ymin:ymax+1, xmin:xmax+1]
            avg_pixel = np.count_nonzero(bbox_pred == 1)
    return image, depth, pred_label, confidence, l, avg_pixel, bbox_conf

def create_frame(pred_dir, pred_merged_dir, _id, recreate=False, bbox_coords=[]):
    canvas_path = os.path.join(pred_merged_dir, _id+'.png')
    image, depth, pred_label, confidence = None, None, None, None
    l = 0.0
    avg_pixel = 0.0
    bbox_conf = None
    if recreate:
        image, depth, pred_label, confidence = read_images(pred_dir, _id)
        if image is None:
            return None, None, None, None, None, None, None, None
        # calculate brightness
        hlsImg = cv2.cvtColor(image, cv2.COLOR_RGB2HLS)
        l = np.average(hlsImg[:,:,1])
        image_title = 'Image (brightness: {:.4f})'.format(l)
        # calculate average pixel value at bbox area
        if bbox_coords:
            ymin, ymax, xmin, xmax = bbox_coords
            bbox_pred = pred_label[ymin:ymax+1, xmin:xmax+1]
            bbox_conf = confidence[ymin:ymax+1, xmin:xmax+1]
            avg_pixel = np.count_nonzero(bbox_pred == 1)
        create_mpl_viz_outputs(canvas_path, image, pred_label, confidence, depth, image_title=image_title, bbox_coords=bbox_coords)
    frame = cv2.imread(canvas_path)
    return frame, image, depth, pred_label, confidence, l, avg_pixel, bbox_conf

def create_diff_frame(pred_merged_dir, _id, image1, depth1, pred_label1, confidence1, 
                      image2, depth2, pred_label2, confidence2, pred_title='prediction', conf_title='confidence'):
    canvas_path = os.path.join(pred_merged_dir, _id+'_diff.png')
    image = np.abs(image1 - image2)
    depth = np.abs(depth1 - depth2)
    pred_label = np.abs(pred_label1 - pred_label2)
    confidence = np.abs(confidence1 - confidence2)
    create_mpl_viz_outputs(canvas_path, image, pred_label, confidence, depth, conf_title=conf_title)
    frame = cv2.imread(canvas_path)
    return frame

def read_raw_image_by_id(data_dir, _id):
    image_path = os.path.join(data_dir, 'images', _id, 'artifact_debayeredrgb_0_'+_id+'.png')
    image = cv2.imread(image_path)
    return image

def read_raw_image_by_row(data_dir, row):
    image = cv2.imread(os.path.join(data_dir, row.artifact_debayeredrgb_0_save_path))
    return image

def read_rectified_image(data_dir, row):
    data_path = os.path.join(data_dir, row.stereo_pipeline_npz_save_path)
    img = np.load(data_path)['left']
    img = normalize_image(img, hdr_mode=row.hdr_mode, return_8_bit=True)
    return cv2.cvtColor(img, cv2.COLOR_RGB2BGR)

def create_video(ids, pred_dir, video_name, read_func=read_saved_frame, fps=2):
    frame = read_func(pred_dir, ids[10])
    height, width, layers = frame.shape
    print(height, width, layers)

    # .avi MJPG,  .mp4 MP4V
    video = cv2.VideoWriter(video_name, cv2.VideoWriter_fourcc(*'MP4V'), fps, (width,height), isColor=True)
    
    good = 0
    for _id in tqdm(ids):
        frame = read_func(pred_dir, _id)
        if frame is not None:
            video.write(frame)
            good += 1
    print('total', len(ids), 'used', good)

    # cv2.destroyAllWindows()
    video.release()

## Create video from raw left images

In [15]:
# create video from raw left images
data_dir = '/data/jupiter/datasets/dust_datasets/Jupiter_1RW9640DKPA085013_20241017_dust_stereo'
# data_dir = '/data/jupiter/datasets/safety_datasets/dust/halo_human_in_dust_day_v5'
# data_dir = '/data3/jupiter/datasets/model_positive/halo_human_w_corn_stubble_0812_oct'
df = pd.read_csv(os.path.join(data_dir, 'annotations.csv'))
# df = pd.read_csv(os.path.join(data_dir, 'master_annotations.csv'))

seq_dfs = get_sequences(df, interval=60, per_camera=False, per_camera_pair=False)  # break the data by intervals between sequences
print(df.shape, len(seq_dfs))

/tmp/ipykernel_1012450/3686842739.py:5: DtypeWarning: Columns (85) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(os.path.join(data_dir, 'annotations.csv'))


(16117, 99) 52


### select images for labeling

#### Combine multiple human in dust test sets

In [6]:
# combine and create datasets for human in dust
# data_root_dir = '/data/jupiter/datasets'
data_root_dir = '/data/jupiter/li.yu/data/dust_data_colletion_for_july_1st'
unlabeled_datasets = [
    "halo_human_in_dust_day_collection_may29",
    "halo_human_in_dust_night_collection_june03",  # for lying down humans
    "halo_human_in_dust_night_collection_june03_2",
    "halo_human_in_dust_day_collection_back_june05",
    "halo_human_in_dust_dusk_collection_front_june07",
    "halo_human_in_dust_night_collection_july29",
]
day_night_splits = ['day', 'dawn_dusk', {'dawn_dusk': list(range(0, 9)), 'night': list(range(9, 19))}, 'day', 'dawn_dusk', 'night']
use_ds = [0, 2, 3, 4, 5]
# split data by day time
image_ids_by_time = {'id':[], 'day_night_split': []}
for di in use_ds:
    dataset = unlabeled_datasets[di]
    df = pd.read_csv(os.path.join(data_root_dir, dataset, 'annotations.csv'))
    df = df.sort_values('collected_on')
    day_night_split = day_night_splits[di]
    if isinstance(day_night_split, str):
        print(dataset, df.shape, day_night_split)
        image_ids_by_time['id'] += df.id.to_list()
        image_ids_by_time['day_night_split'] += [day_night_split] * len(df)
    else:
        seq_dfs = get_sequences(df, interval=60)  # break the data by intervals between sequences
        print(dataset, df.shape, len(seq_dfs), day_night_split)
        for dn_split, seq_ids in day_night_split.items():
            for seq_id in seq_ids:
                seq_df = seq_dfs[seq_id]
                image_ids_by_time['id'] += seq_df.id.to_list()
                image_ids_by_time['day_night_split'] += [dn_split] * len(seq_df)
# save to csv
day_night_split_df = pd.DataFrame(data=image_ids_by_time)
print(day_night_split_df.shape)
day_night_split_df.to_csv('/data/jupiter/li.yu/data/halo_hard_cases/halo_human_in_dust_daynightsplit_for_july01.csv', index=False)

halo_human_in_dust_day_collection_may29 (48102, 108) day
halo_human_in_dust_night_collection_june03_2 (127568, 108) 20 {'dawn_dusk': [0, 1, 2, 3, 4, 5, 6, 7, 8], 'night': [9, 10, 11, 12, 13, 14, 15, 16, 17, 18]}
halo_human_in_dust_day_collection_back_june05 (11675, 108) day


/tmp/ipykernel_2996009/4013714849.py:18: DtypeWarning: Columns (0,1,34,40,49,51,62,75,85,87,89,90,91,99,101) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(os.path.join(data_root_dir, dataset, 'annotations.csv'))


halo_human_in_dust_dusk_collection_front_june07 (36338, 104) dawn_dusk
halo_human_in_dust_night_collection_july29 (7956, 111) night
(228247, 2)


In [27]:
# image_ids_by_time = {'id':[], 'day_night_split': []}
# split data in day/dusk/night
day_night_splits = [{'day': list(range(0,25)), 'dawn_dusk': list(range(25, 31)), 'night': list(range(31, 38))}, 
                   {'dawn_dusk': list(range(0, 14)), 'night': list(range(14, 29))},
                   {'dawn_dusk': [0, 1, 2, 7, 8, 9], 'night': [3, 4, 5, 6, 10, 11, 12]}]
day_night_split = day_night_splits[2]
for dn_split, seq_ids in day_night_split.items():
    for seq_id in seq_ids:
        seq_df = seq_dfs[seq_id]
        image_ids_by_time['id'] += seq_df.id.to_list()
        image_ids_by_time['day_night_split'] += [dn_split] * len(seq_df)
# save to csv
day_night_split_df = pd.DataFrame(data=image_ids_by_time)
print(day_night_split_df.shape)
day_night_split_df.to_csv('/data/jupiter/li.yu/data/halo_hard_cases/halo_vehicle_in_dust_daynightsplit_excludebadnight_for_july01.csv')

(176217, 2)


#### Select human in dust and dust on lens data - each camera is handled separately

In [60]:
def parse_camera_sample_desc(s):
    def parse_camera_desc(cam):
        ts = cam.split(' ')
        if len(ts) == 1:
            return [ts[0], 'normal']
        elif ts[1] not in ['no', 'fix', 'less', 'more']:
            return [ts[0], 'normal', *ts[1:]]
        else:
            return ts
    if len(s) == 0:
        return []
    if not s.endswith(','):
        s += ','
    tokens = s.split('),')
    camera_sample_desc = []
    for token in tokens:
        token = token.strip()
        if not token:
            continue
        seq_i, cam1_cam2 = token.split(' (')
        cam1, cam2 = cam1_cam2.split(', ')
        camera_sample_desc.append([int(seq_i), [parse_camera_desc(cam1), parse_camera_desc(cam2)]])
    return camera_sample_desc

def select_by_camera_sample_ratio(seq_dfs, selected, camera_sample_ratio, save_csv=''):
    def sample(seq_df, cam_desc_list):
        sample_ids = []
        # parse and get cam_df
        if len(cam_desc_list) == 2:
            cam, desc = cam_desc_list
            cam_df = seq_df[seq_df.camera_location == cam]
        elif len(cam_desc_list) == 4:
            cam, desc, before_after, t = cam_desc_list
            if before_after == 'before':
                if len(t) == 19:  # like 2024-11-20T17:29:06
                    t = t + '.999000'
                cam_df = seq_df[(seq_df.camera_location == cam) & (seq_df.collected_on <= t)]
            else:
                cam_df = seq_df[(seq_df.camera_location == cam) & (seq_df.collected_on >= t)]
        else:
            cam, desc, t1, _, t2 = cam_desc_list
            if len(t2) == 19:  # like 2024-11-20T17:29:06
                t2 = t2 + '.999000'
            cam_df = seq_df[(seq_df.camera_location == cam) & (seq_df.collected_on >= t1) & (seq_df.collected_on <= t2)]
        # sampling
        sample_ratio = camera_sample_ratio[desc]
        if sample_ratio < 1:
            sample_df = cam_df.sample(frac=sample_ratio)
        else:
            sample_df = cam_df.sample(n=int(sample_ratio))
        print(pod, seq_i, cam, desc, len(cam_df), len(sample_df))
        sample_ids += sample_df.id.to_list()
        return sample_ids

    parsed_selected = {pod: parse_camera_sample_desc(s) for pod,s in selected.items()}
    selected_ids = []
    for pod, camera_sample_desc in parsed_selected.items():
        for seq_i, two_cam_desc_list in camera_sample_desc:
            seq_df = seq_dfs[seq_i]
            for cam_desc_list in two_cam_desc_list:
                selected_ids += sample(seq_df, cam_desc_list)
    selected_ids = list(set(selected_ids))
    selected_df = pd.DataFrame(data={'id': selected_ids})
    print(selected_df.shape)
    if save_csv:
        selected_df.to_csv(save_csv, index=False)

s = '3 (T05 no, T06 2024-11-19T20:22:23.989000 - 2024-11-19T20:22:58.621000), 4 (T05 fix, T06 before 2024-11-13T21:38:50.904000)'
parse_camera_sample_desc(s)

[[3,
  [['T05', 'no'],
   ['T06',
    'normal',
    '2024-11-19T20:22:23.989000',
    '-',
    '2024-11-19T20:22:58.621000']]],
 [4,
  [['T05', 'fix'], ['T06', 'normal', 'before', '2024-11-13T21:38:50.904000']]]]

In [70]:
# for selection of images for labeling - halo_water_on_ground_jf206_2nd
selected = {'back': '1 (T09 less 2024-11-14T14:38:50 - 2024-11-14T14:39:04, T10 less after 2024-11-14T14:39:14)',
            'front': '1 (T01 2024-11-14T14:38:45 - 2024-11-14T14:38:50, T02 2024-11-14T14:38:45 - 2024-11-14T14:38:54), 3 (T01 no, T02 2024-11-14T14:41:22 - 2024-11-14T14:41:31),', 
            'left': '1 (T13 2024-11-14T14:38:43 - 2024-11-14T14:38:56, T14 2024-11-14T14:38:52 - 2024-11-14T14:39:01)',
            'right': '',}
camera_sample_ratio = {'more': 0.4, 'normal': 0.3, 'less': 0.15, 'fix': 2, 'no': 0}
save_csv = '/data/jupiter/datasets/dust_datasets/halo_water_on_ground_jf206_2nd/selected_for_label.csv'
select_by_camera_sample_ratio(seq_dfs, selected, camera_sample_ratio, save_csv)

back 1 T09 less 45 7
back 1 T10 less 44 7
front 1 T01 normal 18 5
front 1 T02 normal 30 9
front 3 T01 no 75 0
front 3 T02 normal 30 9
left 1 T13 normal 42 13
left 1 T14 normal 30 9
(59, 1)


In [62]:
# for selection of images for labeling - halo_water_on_ground_jf206
selected = {'back': '5 (T09 more 2024-11-20T17:29:06 - 2024-11-20T17:29:09, T10 more 2024-11-20T17:29:04 - 2024-11-20T17:29:13), 5 (T09 no, T10 more 2024-11-20T17:30:08 - 2024-11-20T17:30:14), 5 (T09 no, T10 more 2024-11-20T17:30:28 - 2024-11-20T17:30:40), 5 (T09 no, T10 2024-11-20T17:30:52 - 2024-11-20T17:31:01), 5 (T09 no, T10 2024-11-20T17:33:27 - 2024-11-20T17:33:33)',
            'front': '0 (T01 more, T02 more), 1 (T01 more 2024-11-20T17:21:55 - 2024-11-20T17:22:04, T02 no), 1 (T01 2024-11-20T17:22:17 - 2024-11-20T17:22:30, T02 no), 2 (T01 2024-11-20T17:24:52 - 2024-11-20T17:24:59, T02 no), 5 (T01 before 2024-11-20T17:29:08, T02 more before 2024-11-20T17:29:23), 5 (T01 more 2024-11-20T17:29:57 - 2024-11-20T17:30:09, T02 more 2024-11-20T17:29:57 - 2024-11-20T17:30:09), 5 (T01 more 2024-11-20T17:30:20 - 2024-11-20T17:30:35, T02 2024-11-20T17:30:19 - 2024-11-20T17:30:28), 5 (T01 more 2024-11-20T17:30:42 - 2024-11-20T17:31:12, T02 no), 5 (T01 more after 2024-11-20T17:33:20, T02 no)', 
            'left': '5 (T13 2024-11-20T17:29:01 - 2024-11-20T17:29:07, T14 2024-11-20T17:29:04 - 2024-11-20T17:29:08),',
            'right': '0 (T05 no, T06), 1 (T05 2024-11-20T17:22:01 - 2024-11-20T17:22:08, T06 more 2024-11-20T17:21:56 - 2024-11-20T17:22:05), 2 (T05 2024-11-20T17:25:00 - 2024-11-20T17:25:07, T06 2024-11-20T17:24:58 - 2024-11-20T17:25:00), 5 (T05 2024-11-20T17:30:23 - 2024-11-20T17:31:12, T06 2024-11-20T17:30:23 - 2024-11-20T17:31:12),',}
camera_sample_ratio = {'more': 0.4, 'normal': 0.3, 'less': 0.1, 'fix': 2, 'no': 0}
save_csv = '/data/jupiter/datasets/dust_datasets/halo_water_on_ground_jf206/selected_for_label.csv'
select_by_camera_sample_ratio(seq_dfs, selected, camera_sample_ratio, save_csv)

back 5 T09 more 12 5
back 5 T10 more 30 12
back 5 T09 no 683 0
back 5 T10 more 21 8
back 5 T09 no 683 0
back 5 T10 more 39 16
back 5 T09 no 683 0
back 5 T10 normal 30 9
back 5 T09 no 683 0
back 5 T10 normal 21 6
front 0 T01 more 30 12
front 0 T02 more 30 12
front 1 T01 more 30 12
front 1 T02 no 376 0
front 1 T01 normal 42 13
front 1 T02 no 376 0
front 2 T01 normal 24 7
front 2 T02 no 376 0
front 5 T01 normal 50 15
front 5 T02 more 95 38
front 5 T01 more 39 16
front 5 T02 more 39 16
front 5 T01 more 48 19
front 5 T02 normal 30 9
front 5 T01 more 93 37
front 5 T02 no 836 0
front 5 T01 more 32 13
front 5 T02 no 836 0
left 5 T13 normal 21 6
left 5 T14 normal 15 4
right 0 T05 no 30 0
right 0 T06 normal 30 9
right 1 T05 normal 24 7
right 1 T06 more 30 12
right 2 T05 normal 24 7
right 2 T06 normal 9 3
right 5 T05 normal 150 45
right 5 T06 normal 150 45
(413, 1)


In [34]:
# for selection of images for labeling - halo_human_with_dust_on_lens_jf163_5th
selected = {'back': '0 (T09 less before 2024-11-13T21:38:50.904000, T10 less 2024-11-13T21:38:50.904000 - 2024-11-13T21:39:57.837000), 1 (T09 less 2024-11-13T21:46:18.123000 - 2024-11-13T21:47:26.388000, T10 less 2024-11-13T21:47:26.388000 - 2024-11-13T21:48:15.339000), 2 (T09 less 2024-11-13T22:00:38.928000 - 2024-11-13T22:01:34.872000, T10 less before 2024-11-13T21:56:41.499000), 2 (T09 less after 2024-11-13T22:07:59.821000, T10 less 2024-11-13T22:01:34.872000 - 2024-11-13T22:03:41.080000), 3 (T09 less 2024-11-13T22:14:16.111000 - 2024-11-13T22:15:19.714000, T10 less 2024-11-13T22:15:19.714000 - 2024-11-13T22:16:01.006000), 3 (T09 less 2024-11-13T22:21:19.021000 - 2024-11-13T22:22:45.934000, T10 less 2024-11-13T22:22:45.934000 - 2024-11-13T22:23:46.207000), 4 (T09 less 2024-11-13T22:31:26.081000 - 2024-11-13T22:32:16.364000, T10 less 2024-11-13T22:32:16.364000 - 2024-11-13T22:32:32), 4 (T09 less 2024-11-13T22:32:32 - 2024-11-13T22:32:44, T10 less 2024-11-13T22:32:44 - 2024-11-13T22:33:46), 5 (T09 2024-11-13T22:38:48 - 2024-11-13T22:39:44, T10 2024-11-13T22:39:44 - 2024-11-13T22:40:38), 6 (T09 2024-11-13T23:17:33 - 2024-11-13T23:18:51, T10 2024-11-13T23:18:51 - 2024-11-13T23:19:53), 7 (T09 2024-11-13T23:25:56 - 2024-11-13T23:27:17, T10 2024-11-13T23:27:17 - 2024-11-13T23:28:17),',
            'front': '0 (T01 less 2024-11-13T21:40:17 - 2024-11-13T21:41:35, T02 less 2024-11-13T21:41:36 - 2024-11-13T21:42:45), 1 (T01 less 2024-11-13T21:49:16 - 2024-11-13T21:50:14, T02 less 2024-11-13T21:50:14 - 2024-11-13T21:51:21), 2 (T01 less 2024-11-13T21:57:29 - 2024-11-13T21:58:38, T02 less 2024-11-13T21:58:38 - 2024-11-13T21:59:46), 2 (T01 less 2024-11-13T22:04:37 - 2024-11-13T22:05:42, T02 less 2024-11-13T22:05:42 - 2024-11-13T22:06:36), 3 (T01 less 2024-11-13T22:17:20 - 2024-11-13T22:18:32, T02 less before 2024-11-13T22:14:13), 3 (T01 less after 2024-11-13T22:24:55, T02 less 2024-11-13T22:18:32 - 2024-11-13T22:20:26), 4 (T01 less 2024-11-13T22:28:22 - 2024-11-13T22:29:30, T02 less 2024-11-13T22:29:30 - 2024-11-13T22:30:29), 5 (T01 less 2024-11-13T22:35:44 - 2024-11-13T22:36:54, T02 less 2024-11-13T22:36:54 - 2024-11-13T22:37:48), 6 (T01 2024-11-13T23:20:59 - 2024-11-13T23:22:19, T02 less 2024-11-13T23:22:19 - 2024-11-13T23:23:36), 7 (T01 less 2024-11-13T23:29:24 - 2024-11-13T23:30:14, T02 less 2024-11-13T23:30:14 - 2024-11-13T23:31:27)', 
            'left': '0 (T13 less 2024-11-13T21:41:58 - 2024-11-13T21:43:22, T14 less before 2024-11-13T21:38:12), 1 (T13 less 2024-11-13T21:50:30 - 2024-11-13T21:51:56, T14 less before 2024-11-13T21:46:51), 2 (T13 less 2024-11-13T21:59:07 - 2024-11-13T22:00:11, T14 less 2024-11-13T22:00:11 - 2024-11-13T22:01:10), 2 (T13 less 2024-11-13T22:06:11 - 2024-11-13T22:07:04, T14 less 2024-11-13T22:07:04 - 2024-11-13T22:08:34), 3 (T13 less before 2024-11-13T22:14:11, T14 less 2024-11-13T22:14:11 - 2024-11-13T22:14:51), 3 (T13 less 2024-11-13T22:19:55 - 2024-11-13T22:21:08, T14 less 2024-11-13T22:21:08 - 2024-11-13T22:21:56), 4 (T13 less 2024-11-13T22:29:52 - 2024-11-13T22:31:19, T14 less 2024-11-13T22:31:18 - 2024-11-13T22:32:02), 5 (T13 less 2024-11-13T22:37:10 - 2024-11-13T22:38:15, T14 less 2024-11-13T22:38:15 - 2024-11-13T22:39:36), 6 (T13 less 2024-11-13T23:22:54 - 2024-11-13T23:24:24, T14 less before 2024-11-13T23:18:14), 7 (T13 less 2024-11-13T23:30:44 - 2024-11-13T23:32:03, T14 before 2024-11-13T23:26:29)',
            'right': '0 (T05 less 2024-11-13T21:39:03 - 2024-11-13T21:40:07, T06 less 2024-11-13T21:40:07 - 2024-11-13T21:41:04), 1 (T05 less 2024-11-13T21:47:39 - 2024-11-13T21:48:47, T06 less 2024-11-13T21:48:46 - 2024-11-13T21:49:48), 2 (T05 less 2024-11-13T21:55:59 - 2024-11-13T21:57:15, T06 less 2024-11-13T21:57:15 - 2024-11-13T21:58:07), 2 (T05 less 2024-11-13T22:03:06 - 2024-11-13T22:04:22, T06 less 2024-11-13T22:04:22 - 2024-11-13T22:05:13), 3 (T05 less 2024-11-13T22:15:45 - 2024-11-13T22:16:46, T06 less 2024-11-13T22:16:46 - 2024-11-13T22:17:53), 3 (T05 less 2024-11-13T22:22:55 - 2024-11-13T22:24:22, T06 less 2024-11-13T22:24:22 - 2024-11-13T22:25:30), 4 (T05 less 2024-11-13T22:33:10 - 2024-11-13T22:33:47, T06 less before 2024-11-13T22:28:51), 5 (T05 less 2024-11-13T22:40:11 - 2024-11-13T22:40:46, T06 less before 2024-11-13T22:36:17), 6 (T05 2024-11-13T23:19:03 - 2024-11-13T23:20:30, T06 2024-11-13T23:20:30 - 2024-11-13T23:21:41), 7 (T05 2024-11-13T23:27:30 - 2024-11-13T23:28:49, T06 2024-11-13T23:28:49 - 2024-11-13T23:29:46),',}
camera_sample_ratio = {'normal': 0.33, 'less': 0.15, 'fix': 8, 'no': 2}
save_csv = '/data/jupiter/datasets/dust_datasets/halo_human_with_dust_on_lens_jf163_5th/selected_for_label.csv'
select_by_camera_sample_ratio(seq_dfs, selected, camera_sample_ratio, save_csv)

back 0 T09 less 413 62
back 0 T10 less 127 19
back 1 T09 less 131 20
back 1 T10 less 148 22
back 2 T09 less 169 25
back 2 T10 less 218 33
back 2 T09 less 123 18
back 2 T10 less 206 31
back 3 T09 less 192 29
back 3 T10 less 125 19
back 3 T09 less 187 28
back 3 T10 less 182 27
back 4 T09 less 152 23
back 4 T10 less 47 7
back 4 T09 less 36 5
back 4 T10 less 112 17
back 5 T09 normal 92 30
back 5 T10 normal 162 53
back 6 T09 normal 159 52
back 6 T10 normal 110 36
back 7 T09 normal 168 55
back 7 T10 normal 181 60
front 0 T01 less 84 13
front 0 T02 less 132 20
front 1 T01 less 174 26
front 1 T02 less 126 19
front 2 T01 less 132 20
front 2 T02 less 205 31
front 2 T01 less 121 18
front 2 T02 less 162 24
front 3 T01 less 216 32
front 3 T02 less 231 35
front 3 T01 less 240 36
front 3 T02 less 207 31
front 4 T01 less 204 31
front 4 T02 less 177 27
front 5 T01 less 135 20
front 5 T02 less 162 24
front 6 T01 normal 90 30
front 6 T02 less 232 35
front 7 T01 less 150 22
front 7 T02 less 219 33
left 0 

In [32]:
# for selection of images for labeling - halo_human_with_dust_on_lens_jf163_4th
selected = {'back': '0 (T09 fix, T10 no), 4 (T09 no, T10 fix), 5 (T09 fix, T10 no),',
            'front': '0 (T01 no, T02 no), 1 (T01 less, T02 less), 2 (T01 less, T02 no), 3 (T01 no, T02 no), 5 (T01 no, T02 no),', 
            'left': '0 (T13 fix, T14 less), 1 (T13 less, T14 no), 2 (T13 no, T14 no), 4 (T13 no, T14 no), 5 (T13 no, T14 less),',
            'right': '0 (T05 no, T06 no), 2 (T05 no, T06), 3 (T05, T06 no), 4 (T05, T06 no), 5 (T05 no, T06 no)',}
camera_sample_ratio = {'normal': 0.33, 'less': 0.15, 'fix': 8, 'no': 2}
save_csv = '/data/jupiter/datasets/dust_datasets/halo_human_with_dust_on_lens_jf163_4th/selected_for_label.csv'
select_by_camera_sample_ratio(seq_dfs, selected, camera_sample_ratio, save_csv)

back 0 T09 fix 202 8
back 0 T10 no 202 2
back 4 T09 no 113 2
back 4 T10 fix 113 8
back 5 T09 fix 113 8
back 5 T10 no 113 2
front 0 T01 no 202 2
front 0 T02 no 203 2
front 1 T01 less 224 34
front 1 T02 less 225 34
front 2 T01 less 112 17
front 2 T02 no 112 2
front 3 T01 no 111 2
front 3 T02 no 111 2
front 5 T01 no 113 2
front 5 T02 no 113 2
left 0 T13 fix 203 8
left 0 T14 less 202 30
left 1 T13 less 231 35
left 1 T14 no 231 2
left 2 T13 no 112 2
left 2 T14 no 112 2
left 4 T13 no 113 2
left 4 T14 no 113 2
left 5 T13 no 113 2
left 5 T14 less 113 17
right 0 T05 no 203 2
right 0 T06 no 203 2
right 2 T05 no 112 2
right 2 T06 normal 112 37
right 3 T05 normal 111 37
right 3 T06 no 111 2
right 4 T05 normal 113 37
right 4 T06 no 113 2
right 5 T05 no 113 2
right 5 T06 no 113 2
(356, 1)


In [30]:
# for selection of images for labeling - halo_human_with_dust_on_lens_jf163_3rd
selected = {'back': '0 (T09 less 2024-11-19T19:43:10.348000 - 2024-11-19T19:44:04.627000, T10 less 2024-11-19T19:44:04.627000 - 2024-11-19T19:44:53.578000), 1 (T09 2024-11-19T19:52:10.473000 - 2024-11-19T19:52:19.797000, T10 no), 1 (T09 2024-11-19T19:52:32.118000 - 2024-11-19T19:52:36.780000, T10 2024-11-19T19:52:32.118000 - 2024-11-19T19:52:35.448000), 2 (T09 2024-11-19T19:56:16.560000 - 2024-11-19T19:56:23.553000, T10 2024-11-19T19:56:16.560000 - 2024-11-19T19:56:21.555000), 2 (T09 no, T10 2024-11-19T19:56:32.544000 - 2024-11-19T19:57:09.840000), 3 (T09 less 2024-11-19T23:20:56.929000 - 2024-11-19T23:21:39.220000, T10 less 2024-11-19T23:21:39.220000 - 2024-11-19T23:22:14.185000), 3 (T09 less 2024-11-19T23:25:09.343000 - 2024-11-19T23:25:41.977000, T10 less 2024-11-19T23:25:41.977000 - 2024-11-19T23:26:15.277000), 4 (T09 no, T10 less 2024-11-19T23:42:44.287000 - 2024-11-19T23:42:56.941000), 4 (T09 less 2024-11-19T23:44:56.488000 - 2024-11-19T23:45:33.118000, T10 less after 2024-11-19T23:45:33.118000), 5 (T09 no, T10 less 2024-11-19T23:47:58.972000 - 2024-11-19T23:48:09.295000), 5 (T09 less 2024-11-19T23:50:11.839000 - 2024-11-19T23:50:50.134000, T10 less after 2024-11-19T23:50:50.134000), 6 (T09 less 2024-11-19T23:54:21.256000 - 2024-11-19T23:55:23.860000, T10 less 2024-11-19T23:55:23.860000 - 2024-11-19T23:55:53.165000), 6 (T09 less 2024-11-19T23:58:04.700000 - 2024-11-19T23:58:12.359000, T10 no), 6 (T09 less 2024-11-20T00:00:12.239000 - 2024-11-20T00:00:55.862000, T10 less 2024-11-20T00:00:55.862000 - 2024-11-20T00:01:44.147000), 7 (T09 less, T10 less),',
            'front': '1 (T01 less 2024-11-19T19:52:20.128000 - 2024-11-19T19:52:30.118000, T02 less 2024-11-19T19:52:20.128000 - 2024-11-19T19:52:30.118000), 2 (T01 no, T02 less 2024-11-19T19:56:00.907000 - 2024-11-19T19:56:13.228000), 3 (T01 fix, T02 fix), 4 (T01 less 2024-11-19T23:43:18.922000 - 2024-11-19T23:44:06.874000, T02 less 2024-11-19T23:44:06.874000 - 2024-11-19T23:44:36.844000), 5 (T01 less 2024-11-19T23:48:32.941000 - 2024-11-19T23:49:16.231000, T02 less 2024-11-19T23:49:16.231000 - 2024-11-19T23:49:49.864000),', 
            'left': '0 (T13 fix after 2024-11-19T19:46:50.461000, T14 less before 2024-11-19T19:43:51.640000), 1 (T13 2024-11-19T19:52:37.113000 - 2024-11-19T19:52:43.437000, T14 2024-11-19T19:52:37.113000 - 2024-11-19T19:52:43.437000), 3 (T13 less 2024-11-19T23:19:52.327000 - 2024-11-19T23:20:46.606000, T14 less 2024-11-19T23:20:46.606000 - 2024-11-19T23:21:24.901000), 3 (T13 less 2024-11-19T23:24:12.067000 - 2024-11-19T23:24:58.021000, T14 less 2024-11-19T23:24:58.021000 - 2024-11-19T23:25:29.656000), 4 (T13 less 2024-11-19T23:44:13.865000 - 2024-11-19T23:44:49.495000, T14 less 2024-11-19T23:44:49.495000 - 2024-11-19T23:45:16.135000), 5 (T13 less 2024-11-19T23:49:23.887000 - 2024-11-19T23:50:10.507000, T14 less 2024-11-19T23:50:07.843000 - 2024-11-19T23:50:44.806000), 6 (T13 less before 2024-11-19T23:54:03.607000, T14 less 2024-11-19T23:54:03.607000 - 2024-11-19T23:54:48.562000), 6 (T13 less 2024-11-19T23:57:24.074000 - 2024-11-19T23:57:51.713000, T14 less 2024-11-19T23:57:44.720000 - 2024-11-19T23:58:15.689000), 6 (T13 less 2024-11-19T23:59:04.640000 - 2024-11-19T23:59:51.593000, T14 less 2024-11-19T23:59:51.593000 - 2024-11-20T00:00:40.211000), 6 (T13 less after 2024-11-20T00:02:45.752000, T14 no),',
            'right': '3 (T05 less 2024-11-19T23:21:57.871000 - 2024-11-19T23:22:30.505000, T06 less 2024-11-19T23:22:30.505000 - 2024-11-19T23:23:41.101000), 3 (T05 less 2024-11-19T23:25:59.962000 - 2024-11-19T23:26:29.266000, T06 less after 2024-11-19T23:26:29.266000), 4 (T05 less before 2024-11-19T23:43:03.937000, T06 less 2024-11-19T23:43:03.937000 - 2024-11-19T23:43:44.563000), 4 (T05 less 2024-11-19T23:45:47.440000 - 2024-11-19T23:46:12.082000, T06 no), 5 (T05 less 2024-11-19T23:47:57.976000 - 2024-11-19T23:48:15.625000, T06 less 2024-11-19T23:48:15.625000 - 2024-11-19T23:48:55.918000), 5 (T05 less 2024-11-19T23:51:13.780000 - 2024-11-19T23:51:40.420000, T06 no), 6 (T05 less 2024-11-19T23:55:28.525000 - 2024-11-19T23:56:15.145000, T06 less 2024-11-19T23:56:15.145000 - 2024-11-19T23:56:33.793000), 6 (T05 less 2024-11-20T00:01:13.513000 - 2024-11-20T00:01:50.476000, T06 less 2024-11-20T00:01:50.476000 - 2024-11-20T00:02:05.461000)',}
camera_sample_ratio = {'normal': 0.33, 'less': 0.15, 'fix': 8, 'no': 2}
save_csv = '/data/jupiter/datasets/dust_datasets/halo_human_with_dust_on_lens_jf163_3rd/selected_for_label.csv'
select_by_camera_sample_ratio(seq_dfs, selected, camera_sample_ratio, save_csv)

back 0 T09 less 164 25
back 0 T10 less 148 22
back 1 T09 normal 20 7
back 1 T10 no 454 2
back 1 T09 normal 15 5
back 1 T10 normal 11 4
back 2 T09 normal 22 7
back 2 T10 normal 16 5
back 2 T09 no 282 2
back 2 T10 normal 16 5
back 3 T09 less 128 19
back 3 T10 less 106 16
back 3 T09 less 99 15
back 3 T10 less 101 15
back 4 T09 no 662 2
back 4 T10 less 39 6
back 4 T09 less 111 17
back 4 T10 less 135 20
back 5 T09 no 717 2
back 5 T10 less 32 5
back 5 T09 less 116 17
back 5 T10 less 160 24
back 6 T09 less 114 17
back 6 T10 less 89 13
back 6 T09 less 24 4
back 6 T10 no 1529 2
back 6 T09 less 132 20
back 6 T10 less 146 22
back 7 T09 less 411 62
back 7 T10 less 411 62
front 1 T01 less 31 5
front 1 T02 less 31 5
front 2 T01 no 282 2
front 2 T02 less 38 6
front 3 T01 fix 1391 8
front 3 T02 fix 1391 8
front 4 T01 less 145 22
front 4 T02 less 91 14
front 5 T01 less 131 20
front 5 T02 less 102 15
left 0 T13 fix 137 8
left 0 T14 less 214 32
left 1 T13 normal 19 6
left 1 T14 normal 19 6
left 3 T13 les

In [26]:
# for selection of images for labeling - halo_human_in_dust_jf149_4th
selected = {'back': '3 (T09 no, T10 after 2024-11-07T21:34:08.247000), 4 (T09, T10 no), 10 (T09 fix, T10), 11 (T09, T10 fix),',
            'front': '0 (T01 no, T02), 2 (T01 fix, T02 fix), 3 (T01 no, T02 no), 6 (T01 no, T02 no), 7 (T01 no, T02), 8 (T01, T02 less), 9 (T01 less, T02 less), 11 (T01 no, T02 no)', 
            'left': '0 (T13 before 2024-11-07T21:17:19.592000, T14 no), 5 (T13 no, T14), 6 (T13 fix, T14 no), 7 (T13, T14 no), 10 (T13 no, T14 fix), 11 (T13 fix, T14 less), 12 (T13 less, T14 less), 13 (T13 less, T14 less),',
            'right': '0 (T05 no, T06 no), 1 (T05 no, T06), 2 (T05 less, T06 less), 4 (T05 no, T06 less), 6 (T05 no, T06 no), 8 (T05 no, T06 less), 10 (T05 no, T06 less), 12 (T05 no, T06 no)',}
camera_sample_ratio = {'normal': 0.33, 'less': 0.15, 'fix': 8, 'no': 2}
save_csv = '/data/jupiter/datasets/dust_datasets/halo_human_in_dust_jf149_4th/selected_for_label.csv'
select_by_camera_sample_ratio(seq_dfs, selected, camera_sample_ratio, save_csv)

back 3 T09 no 112 2
back 3 T10 normal 93 31
back 4 T09 normal 112 37
back 4 T10 no 112 2
back 10 T09 fix 111 8
back 10 T10 normal 111 37
back 11 T09 normal 187 62
back 11 T10 fix 187 8
front 0 T01 no 224 2
front 0 T02 normal 223 74
front 2 T01 fix 111 8
front 2 T02 fix 111 8
front 3 T01 no 112 2
front 3 T02 no 112 2
front 6 T01 no 113 2
front 6 T02 no 113 2
front 7 T01 no 222 2
front 7 T02 normal 223 74
front 8 T01 normal 112 37
front 8 T02 less 110 16
front 9 T01 less 112 17
front 9 T02 less 112 17
front 11 T01 no 187 2
front 11 T02 no 187 2
left 0 T13 normal 110 36
left 0 T14 no 223 2
left 5 T13 no 112 2
left 5 T14 normal 112 37
left 6 T13 fix 112 8
left 6 T14 no 113 2
left 7 T13 normal 223 74
left 7 T14 no 222 2
left 10 T13 no 111 2
left 10 T14 fix 111 8
left 11 T13 fix 187 8
left 11 T14 less 187 28
left 12 T13 less 112 17
left 12 T14 less 112 17
left 13 T13 less 111 17
left 13 T14 less 111 17
right 0 T05 no 224 2
right 0 T06 no 223 2
right 1 T05 no 110 2
right 1 T06 normal 110 36
r

In [24]:
# for selection of images for labeling - halo_human_in_dust_jf149_3rd
selected = {'back': '0 (T09 2024-11-19T20:05:37.997000 - 2024-11-19T20:05:49.319000, T10 no), 1 (T09 no, T10 no), 2 (T09 less 2024-11-19T20:13:17.536000 - 2024-11-19T20:14:26.467000, T10 less 2024-11-19T20:12:50.563000 - 2024-11-19T20:13:17.536000), 3 (T09 2024-11-19T20:20:22.444000 - 2024-11-19T20:21:26.713000, T10 less 2024-11-19T20:21:26.713000 - 2024-11-19T20:21:59.680000), 4 (T09 less 2024-11-19T20:31:43.329000 - 2024-11-19T20:32:03.076000, T10 2024-11-19T20:32:03.076000 - 2024-11-19T20:32:27.052000), 5 (T09 less 2024-11-19T20:38:34.019000 - 2024-11-19T20:38:50.003000, T10 2024-11-19T20:38:50.003000 - 2024-11-19T20:39:16.643000), 6 (T09 2024-11-19T20:57:55.189000 - 2024-11-19T20:58:10.507000, T10 2024-11-19T20:58:10.507000 - 2024-11-19T20:58:42.142000), 7 (T09 less 2024-11-19T21:03:48.501000 - 2024-11-19T21:04:12.810000, T10 2024-11-19T21:03:22.527000 - 2024-11-19T21:03:48.501000), 8 (T09 less 2024-11-19T21:10:17.445000 - 2024-11-19T21:10:59.403000, T10 fix), 9 (T09 less 2024-11-19T21:16:13.422000 - 2024-11-19T21:17:00.375000, T10 2024-11-19T21:17:00.375000 - 2024-11-19T21:17:32.343000), 10 (T09 less 2024-11-19T23:09:33.948000 - 2024-11-19T23:10:24.897000, T10 2024-11-19T23:10:24.897000 - 2024-11-19T23:11:01.194000), 11 (T09 less 2024-11-19T23:15:28.925000 - 2024-11-19T23:16:34.526000, T10 less 2024-11-19T23:16:34.526000 - 2024-11-19T23:16:57.170000), 12 (T09 2024-11-19T23:36:35.323000 - 2024-11-19T23:36:51.640000, T10 2024-11-19T23:36:51.640000 - 2024-11-19T23:37:31.600000), 13 (T09 less 2024-11-19T23:40:58.060000 - 2024-11-19T23:41:15.043000, T10 less 2024-11-19T23:41:15.043000 - 2024-11-19T23:41:44.680000), 14 (T09 2024-11-20T00:11:02.589000 - 2024-11-20T00:11:10.581000, T10 2024-11-20T00:11:10.581000 - 2024-11-20T00:11:49.543000), 14 (T09 2024-11-20T00:16:36.256000 - 2024-11-20T00:17:02.563000, T10 2024-11-20T00:17:02.563000 - 2024-11-20T00:17:26.206000), 15 (T09 2024-11-20T00:25:39.714000 - 2024-11-20T00:26:04.356000, T10 2024-11-20T00:26:04.356000 - 2024-11-20T00:26:29.664000), 15 (T09 2024-11-20T00:29:07.173000 - 2024-11-20T00:29:41.805000, T10 2024-11-20T00:29:41.805000 - 2024-11-20T00:30:13.773000)',
            'front': '1 (T01 fix, T02 less), 3 (T01 no, T02 after 2024-11-19T20:23:52.234000), 4 (T01 after 2024-11-19T20:33:25.327000, T02 after 2024-11-19T20:34:50.908000), 5 (T01 fix, T02 fix), 6 (T01 less after 2024-11-19T21:00:16.048000, T02 less after 2024-11-19T21:00:56.674000), 7 (T01 fix, T02 no), 8 (T01 no, T02 less after 2024-11-19T21:13:18.598000), 9 (T01 less after 2024-11-19T21:18:50.266000, T02 after 2024-11-19T21:19:20.236000), 10 (T01 less before 2024-11-19T23:08:52.657000, T02 less before 2024-11-19T23:08:52.657000), 11 (T01 less before 2024-11-19T23:15:10.279000, T02 less before 2024-11-19T23:15:10.279000), 12 (T01 after 2024-11-19T23:34:56.092000, T02 after 2024-11-19T23:34:56.092000), 13 (T01 2024-11-19T23:39:32.815000 - 2024-11-19T23:39:53.128000, T02 2024-11-19T23:39:54.127000 - 2024-11-19T23:40:10.777000), 14 (T01 2024-11-19T00:12:23.842000 - 2024-11-20T00:12:53.812000, T02 less 2024-11-20T00:12:53.812000 - 2024-11-20T00:13:38.767000), 14 (T01 less 2024-11-20T00:14:39.373000 - 2024-11-20T00:14:55.024000, T02 2024-11-20T00:14:55.024000 - 2024-11-20T00:15:31.987000), 15 (T01 less 2024-11-20T00:30:36.082000 - 2024-11-20T00:31:04.054000, T02 2024-11-20T00:27:24.607000 - 2024-11-20T00:28:42.196000), 15 (T01 no, T02 after 2024-11-20T00:31:03.721000)',
            'left': '0 (T13 2024-11-19T20:05:51.317000 - 2024-11-19T20:06:08.633000, T14 before 2024-11-19T20:05:51.317000), 1 (T13 no, T14 no), 2 (T13 2024-11-19T20:14:34.459000 - 2024-11-19T20:14:53.107000, T14 less 2024-11-19T20:14:01.825000 - 2024-11-19T20:14:34.459000), 3 (T13 before 2024-11-19T20:20:15.784000, T14 2024-11-19T20:20:15.784000 - 2024-11-19T20:20:41.758000), 4 (T13 less before 2024-11-19T20:30:48.151000, T14 2024-11-19T20:30:48.151000 - 2024-11-19T20:31:10.795000), 4 (T13 less after 2024-11-19T20:34:45.580000, T14 no), 5 (T13 before 2024-11-19T20:37:46.400000, T14 2024-11-19T20:37:46.400000 - 2024-11-19T20:38:22.697000), 5 (T13 less after 2024-11-19T20:41:24.182000, T14 no), 6 (T13 less 2024-11-19T20:56:58.246000 - 2024-11-19T20:57:11.566000, T14 2024-11-19T20:57:11.566000 - 2024-11-19T20:57:46.864000), 7 (T13 less 2024-11-19T21:04:41.115000 - 2024-11-19T21:05:36.393000, T14 less 2024-11-19T21:04:16.473000 - 2024-11-19T21:04:41.115000), 8 (T13 fix before 2024-11-19T21:10:11.118000, T14 less 2024-11-19T21:10:11.118000 - 2024-11-19T21:10:40.422000), 8 (T13 less after 2024-11-19T21:13:25.923000, T14 no), 9 (T13 less before 2024-11-19T21:16:05.097000, T14 less 2024-11-19T21:16:05.097000 - 2024-11-19T21:16:30.072000), 9 (T13 less after 2024-11-19T21:20:22.172000, T14 no), 10 (T13 2024-11-19T23:08:49.326000 - 2024-11-19T23:09:23.625000, T14 less 2024-11-19T23:09:23.625000 - 2024-11-19T23:09:44.604000), 11 (T13 before 2024-11-19T23:15:18.935000, T14 less 2024-11-19T23:15:18.935000 - 2024-11-19T23:15:54.566000), 11 (T13 fix after 2024-11-19T23:15:18.935000, T14 no), 12 (T13 less, T14), 13 (T13 less 2024-11-19T23:39:43.468000 - 2024-11-19T23:40:50.068000, T14 less 2024-11-19T23:40:50.068000 - 2024-11-19T23:41:16.042000), 14 (T13 less 2024-11-20T00:09:58.320000 - 2024-11-20T00:10:14.970000, T14 less 2024-11-20T00:10:14.970000 - 2024-11-20T00:10:51.600000), 14 (T13 less 2024-11-20T00:13:18.787000 - 2024-11-20T00:15:50.635000, T14 2024-11-20T00:15:50.635000 - 2024-11-20T00:16:41.251000), 15 (T13 less before 2024-11-20T00:25:14.406000, T14 less 2024-11-20T00:25:14.406000 - 2024-11-20T00:25:54.366000), 15 (T13 less 2024-11-20T00:28:07.566000 - 2024-11-20T00:28:53.520000, T14 no), 15 (T13 less after 2024-11-20T00:31:27.034000, T14 no),',
            'right': '3 (T05 no, T06 2024-11-19T20:22:23.989000 - 2024-11-19T20:22:58.621000), 4 (T05 fix, T06 no), 5 (T05 less 2024-11-19T20:39:01.990000 - 2024-11-19T20:39:47.278000, T06 less 2024-11-19T20:39:47.278000 - 2024-11-19T20:40:02.929000), 6 (T05 no, T06 2024-11-19T20:59:29.095000 - 2024-11-19T20:59:59.065000), 7 (T05 2024-11-19T21:02:52.225000 - 2024-11-19T21:03:10.873000, T06 before 2024-11-19T21:02:52.225000), 8 (T05 less 2024-11-19T21:11:37.699000 - 2024-11-19T21:11:53.350000, T06 less 2024-11-19T21:11:53.350000 - 2024-11-19T21:12:24.652000), 9 (T05 no, T06 less 2024-11-19T21:18:21.628000 - 2024-11-19T21:18:33.283000), 10 (T05 less 2024-11-19T23:10:44.878000 - 2024-11-19T23:11:17.845000, T06 less after 2024-11-19T23:11:17.845000), 11 (T05 less 2024-11-19T23:16:33.529000 - 2024-11-19T23:17:09.160000, T06 less after 2024-11-19T23:17:09.160000), 12 (T05 2024-11-19T23:34:08.473000 - 2024-11-19T23:34:19.129000, T06 2024-11-19T23:34:24.457000 - 2024-11-19T23:34:49.765000), 13 (T05 2024-11-19T23:38:49.525000 - 2024-11-19T23:39:05.509000, T06 2024-11-19T23:39:05.509000 - 2024-11-19T23:39:31.483000), 14 (T05 less 2024-11-20T00:11:38.554000 - 2024-11-20T00:11:47.878000, T06 less 2024-11-20T00:11:47.878000 - 2024-11-20T00:12:46.486000), 14 (T05 less 2024-11-20T00:17:07.891000 - 2024-11-20T00:17:48.184000, T06 less after 2024-11-20T00:17:44.188000), 15 (T05 less 2024-11-20T00:26:18.340000 - 2024-11-20T00:26:36.655000, T06 less 2024-11-20T00:26:36.655000 - 2024-11-20T00:27:09.289000), 15 (T05 less 2024-11-20T00:29:55.456000 - 2024-11-20T00:30:22.429000, T06 less 2024-11-20T00:30:22.429000 - 2024-11-20T00:30:55.396000)',}
camera_sample_ratio = {'normal': 0.33, 'less': 0.15, 'fix': 8, 'no': 2}
save_csv = '/data/jupiter/datasets/dust_datasets/halo_human_in_dust_jf149_3rd/selected_for_label.csv'
select_by_camera_sample_ratio(seq_dfs, selected, camera_sample_ratio, save_csv)

back 0 T09 normal 35 12
back 0 T10 no 233 2
back 1 T09 no 167 2
back 1 T10 no 167 2
back 2 T09 less 133 20
back 2 T10 less 82 12
back 3 T09 normal 118 39
back 3 T10 less 100 15
back 4 T09 less 60 9
back 4 T10 normal 73 24
back 5 T09 less 49 7
back 5 T10 normal 81 27
back 6 T09 normal 47 16
back 6 T10 normal 96 32
back 7 T09 less 74 11
back 7 T10 normal 79 26
back 8 T09 less 127 19
back 8 T10 fix 602 8
back 9 T09 less 142 21
back 9 T10 normal 97 32
back 10 T09 less 79 12
back 10 T10 normal 110 36
back 11 T09 less 123 18
back 11 T10 less 69 10
back 12 T09 normal 50 16
back 12 T10 normal 121 40
back 13 T09 less 52 8
back 13 T10 less 90 14
back 14 T09 normal 25 8
back 14 T10 normal 118 39
back 14 T09 normal 80 26
back 14 T10 normal 72 24
back 15 T09 normal 75 25
back 15 T10 normal 77 25
back 15 T09 normal 29 10
back 15 T10 normal 97 32
front 1 T01 fix 167 8
front 1 T02 less 167 25
front 3 T01 no 746 2
front 3 T02 normal 143 47
front 4 T01 normal 346 114
front 4 T02 normal 89 29
front 5 T01

#### Select human in dust and dust on lens data - handle two cameras together

In [24]:
def parse_camera_sample_desc(s):
    def parse_camera_desc(cam):
        ts = cam.split(' ')
        if len(ts) == 1:
            return [ts[0], 'normal']
        else:
            return ts
    if not s.endswith(','):
        s += ','
    tokens = s.split('),')
    camera_sample_desc = {}
    for token in tokens:
        token = token.strip()
        if not token:
            continue
        seq_i, cam1_cam2 = token.split(' (')
        cam1, cam2 = cam1_cam2.split(', ')
        camera_sample_desc[int(seq_i)] = [parse_camera_desc(cam1), parse_camera_desc(cam2)]
    return camera_sample_desc

def select_by_camera_sample_ratio(seq_dfs, selected, camera_sample_ratio, save_csv=''):
    def sample(seq_df, cam_desc_list):
        sample_ids = []
        for cam, desc in cam_desc_list:
            cam_df = seq_df[seq_df.camera_location == cam]
            sample_ratio = camera_sample_ratio[desc]
            if sample_ratio < 1:
                sample_df = cam_df.sample(frac=sample_ratio)
            else:
                sample_df = cam_df.sample(n=int(sample_ratio))
            print(pod, seq_i, cam, desc, len(cam_df), len(sample_df))
            sample_ids += sample_df.id.to_list()
        return sample_ids
    def prepare_new_seq_df_n_cam_desc(seq_df, t, after, cam_i, cam_desc_list):
        if after:
            sub_seq_df = seq_df[seq_df.collected_on >= t]
        else:
            sub_seq_df = seq_df[seq_df.collected_on < t]
        s = 'normal' if len(cam_desc_list[cam_i]) == 3 else cam_desc_list[cam_i][1]
        cam_desc_list_copy = [[cam_desc_list[cam_i][0], s]]
        return sub_seq_df, cam_desc_list_copy
    parsed_selected = {pod: parse_camera_sample_desc(s) for pod,s in selected.items()}
    selected_ids = []
    for pod, camera_sample_desc in parsed_selected.items():
        for seq_i, cam_desc_list in camera_sample_desc.items():
            seq_df = seq_dfs[seq_i]
            # handle splitting a sequence
            cam_i = -1
            if 'after' in cam_desc_list[0]:
                cam_i = 0
            elif 'after' in cam_desc_list[1]:
                cam_i = 1
            if cam_i >= 0:
                t = cam_desc_list[cam_i][cam_desc_list[cam_i].index('after') + 1]
                # do sampling on first split for cam_i
                sub_seq_df, cam_desc_list_copy = prepare_new_seq_df_n_cam_desc(seq_df, t, False, 1-cam_i, cam_desc_list)
                print(pod, seq_i, 'first split', cam_desc_list_copy)
                selected_ids += sample(sub_seq_df, cam_desc_list_copy)
                # do sampling for second split for cam_i
                sub_seq_df, cam_desc_list_copy = prepare_new_seq_df_n_cam_desc(seq_df, t, True, cam_i, cam_desc_list)
                print(pod, seq_i, 'second split', cam_desc_list_copy)
                selected_ids += sample(sub_seq_df, cam_desc_list_copy)
            else:
                selected_ids += sample(seq_df, cam_desc_list)
    selected_ids = list(set(selected_ids))
    selected_df = pd.DataFrame(data={'id': selected_ids})
    print(selected_df.shape)
    if save_csv:
        selected_df.to_csv(save_csv, index=False)

s = "4 (T09 less, T10 after 2024-11-05T21:54:52.509000), 10 (T09 less, T10 no), 11 (T09, T10 no), 12 (T09 less, T10 less), 13 (T09, T10 less)"
print(parse_camera_sample_desc(s))

{4: [['T09', 'less'], ['T10', 'after', '2024-11-05T21:54:52.509000']], 10: [['T09', 'less'], ['T10', 'no']], 11: [['T09', 'normal'], ['T10', 'no']], 12: [['T09', 'less'], ['T10', 'less']], 13: [['T09', 'normal'], ['T10', 'less']]}


In [26]:
# for selection of images for labeling - halo_human_in_dust_jf149_2nd
selected = {'back': '4 (T09 less, T10 after 2024-11-05T21:54:52.509000), 10 (T09 less, T10 no), 11 (T09, T10 no), 12 (T09 less, T10 less), 13 (T09, T10 less)',
            'front': '8 (T01 no, T02 less), 16 (T01 fix, T02), 17 (T01 less, T02 less)',
            'left': '0 (T13 no, T14 no), 1 (T13, T14 fix), 2 (T13 fix, T14), 5 (T13 less, T14 fix), 6 (T13 no, T14 no), 7 (T13 no, T14), 14 (T13 no, T14), 15 (T13 less, T14 less)',
            'right': '9 (T05 fix, T06 fix), 18 (T05 fix, T06), 19 (T05, T06 no)',}
camera_sample_ratio = {'normal': 0.33, 'less': 0.15, 'fix': 8, 'no': 2}
save_csv = '/data/jupiter/datasets/dust_datasets/halo_human_in_dust_jf149_2nd/selected_for_label.csv'
select_by_camera_sample_ratio(seq_dfs, selected, camera_sample_ratio, save_csv)

back 4 first split [['T09', 'less']]
back 4 T09 less 191 29
back 4 second split [['T10', 'normal']]
back 4 T10 normal 111 37
back 10 T09 less 40 6
back 10 T10 no 40 2
back 11 T09 normal 191 63
back 11 T10 no 190 2
back 12 T09 less 188 28
back 12 T10 less 188 28
back 13 T09 normal 111 37
back 13 T10 less 111 17
front 8 T01 no 409 2
front 8 T02 less 409 61
front 16 T01 fix 187 8
front 16 T02 normal 187 62
front 17 T01 less 199 30
front 17 T02 less 199 30
left 0 T13 no 37 2
left 0 T14 no 37 2
left 1 T13 normal 115 38
left 1 T14 fix 114 8
left 2 T13 fix 81 8
left 2 T14 normal 82 27
left 5 T13 less 18 3
left 5 T14 fix 18 8
left 6 T13 no 27 2
left 6 T14 no 27 2
left 7 T13 no 24 2
left 7 T14 normal 24 8
left 14 T13 no 113 2
left 14 T14 normal 113 37
left 15 T13 less 185 28
left 15 T14 less 187 28
right 9 T05 fix 228 8
right 9 T06 fix 229 8
right 18 T05 fix 206 8
right 18 T06 normal 206 68
right 19 T05 normal 117 39
right 19 T06 no 117 2
(780, 1)


In [28]:
# for selection of images for labeling - halo_human_with_dust_on_lens_jf163_2nd
selected = {'back': '2 (T09 less after 2024-10-30T20:54:41.005000, T10 less), 6 (T09 less after 2024-10-30T22:08:28.240000, T10 less), 11 (T09 after 2024-10-30T23:51:36.028000, T10 less)',
            'front': '0 (T01 less after 2024-10-30T20:46:25.501000, T02 less), 4 (T01 less after 2024-10-30T22:01:01.687000, T02 less), 9 (T01 less after 2024-10-30T23:42:22.588000, T02 less),',
            'left': '3 (T13 less after 2024-10-30T20:58:20.451000, T14 less), 7 (T13 less after 2024-10-30T22:12:08.686000, T14 less), 8 (T13 after 2024-10-30T23:38:51.802000, T14)',
            'right': '1 (T05 less after 2024-10-30T20:50:40.246000, T06), 5 (T05 less after 2024-10-30T22:05:10.105000, T06), 10 (T05 after 2024-10-30T23:47:23.287000, T06)',}
camera_sample_ratio = {'normal': 0.33, 'less': 0.15, 'fix': 8, 'no': 2}
save_csv = '/data/jupiter/datasets/dust_datasets/halo_human_with_dust_on_lens_jf163_2nd/selected_for_label.csv'
select_by_camera_sample_ratio(seq_dfs, selected, camera_sample_ratio, save_csv)

back 2 first split [['T10', 'less']]
back 2 T10 less 227 34
back 2 second split [['T09', 'less']]
back 2 T09 less 156 23
back 6 first split [['T10', 'less']]
back 6 T10 less 158 24
back 6 second split [['T09', 'less']]
back 6 T09 less 224 34
back 11 first split [['T10', 'less']]
back 11 T10 less 227 34
back 11 second split [['T09', 'normal']]
back 11 T09 normal 292 96
front 0 first split [['T02', 'less']]
front 0 T02 less 250 38
front 0 second split [['T01', 'less']]
front 0 T01 less 216 32
front 4 first split [['T02', 'less']]
front 4 T02 less 228 34
front 4 second split [['T01', 'less']]
front 4 T01 less 229 34
front 9 first split [['T02', 'less']]
front 9 T02 less 156 23
front 9 second split [['T01', 'less']]
front 9 T01 less 148 22
left 3 first split [['T14', 'less']]
left 3 T14 less 252 38
left 3 second split [['T13', 'less']]
left 3 T13 less 207 31
left 7 first split [['T14', 'less']]
left 7 T14 less 174 26
left 7 second split [['T13', 'less']]
left 7 T13 less 208 31
left 8 first

In [24]:
# for selection of images for labeling - halo_human_in_dust_jf149
selected = {'back': '3 (T10 less, T09 no), 4 (T09 less, T10 no), 12 (T09 fix, T10 fix), 13 (T09 fix, T10 no), 14 (T09 fix, T10 no)',
            'front': '0 (T01 less, T02 no), 7 (T02 less, T01 no), 8 (T02 less, T01 no), 9 (T01 less, T02 no), 10 (T01 less, T02 no)',
            'left': '5 (T14 fix, T13 no), 6 (T13 less, T14 no), 15 (T13 less, T14 no)',
            'right': '1 (T06 less, T05 no), 2 (T05 less, T06 no), 11 (T05 fix, T06 no)',}
camera_sample_ratio = {'normal': 0.33, 'less': 0.15, 'fix': 5, 'no': 2}
save_csv = '/data/jupiter/datasets/dust_datasets/halo_human_in_dust_jf149/selected_for_label.csv'
select_by_camera_sample_ratio(seq_dfs, selected, camera_sample_ratio, save_csv)

back 3 T10 less 188 28
back 3 T09 no 188 2
back 4 T09 less 111 17
back 4 T10 no 111 2
back 12 T09 fix 120 5
back 12 T10 fix 120 5
back 13 T09 fix 190 5
back 13 T10 no 189 2
back 14 T09 fix 112 5
back 14 T10 no 111 2
front 0 T01 less 111 17
front 0 T02 no 111 2
front 7 T02 less 113 17
front 7 T01 no 113 2
front 8 T02 less 111 17
front 8 T01 no 111 2
front 9 T01 less 185 28
front 9 T02 no 186 2
front 10 T01 less 111 17
front 10 T02 no 111 2
left 5 T14 fix 194 5
left 5 T13 no 194 2
left 6 T13 less 188 28
left 6 T14 no 188 2
left 15 T13 less 167 25
left 15 T14 no 167 2
right 1 T06 less 186 28
right 1 T05 no 186 2
right 2 T05 less 111 17
right 2 T06 no 111 2
right 11 T05 fix 186 5
right 11 T06 no 186 2
(299, 1)


In [40]:
# for selection of images for labeling - halo_human_with_dust_on_lens_jf163
selected = {'back': '4 (T10 less, T09 no), 5 (T09 less, T10 no), 10 (T10 less, T09 less)',
            'front': '0 (T02 less, T01 no), 1 (T01 less, T02 no), 8 (T02 less, T01 less), 12 (T02 less, T01 less),',
            'left': '6 (T14 less, T13 less), 7 (T14 less, T13 less), 11 (T14 less, T13 less)',
            'right': '2 (T06 less, T05 no), 3 (T05 less, T06 no), 9 (T06 less, T05 less)',}
camera_sample_ratio = {'normal': 0.33, 'less': 0.12, 'fix': 5, 'no': 2}
save_csv = '/data/jupiter/datasets/dust_datasets/halo_human_with_dust_on_lens_jf163/selected_for_label.csv'
select_by_camera_sample_ratio(seq_dfs, selected, camera_sample_ratio, save_csv)

back 4 T10 less 169 20
back 4 T09 no 168 2
back 5 T09 less 60 7
back 5 T10 no 60 2
back 10 T10 less 382 46
back 10 T09 less 380 46
front 0 T02 less 112 13
front 0 T01 no 112 2
front 1 T01 less 111 13
front 1 T02 no 111 2
front 8 T02 less 200 24
front 8 T01 less 200 24
front 12 T02 less 233 28
front 12 T01 less 232 28
left 6 T14 less 298 36
left 6 T13 less 296 36
left 7 T14 less 248 30
left 7 T13 less 248 30
left 11 T14 less 384 46
left 11 T13 less 383 46
right 2 T06 less 186 22
right 2 T05 no 186 2
right 3 T05 less 116 14
right 3 T06 no 116 2
right 9 T06 less 762 91
right 9 T05 less 764 92
(704, 1)


In [35]:
# for selection of images for labeling - halo_dust_smudge_JUPD-1105_2024-10-23
selected = {'back': '34 (T10 less, T09 no), 35 (T09 less, T10 no), 38 (T10, T09 no), 39 (T09, T10 less),',
            'front': '8 (T01 less, T02 no), 9 (T02 less, T01 no), 10 (T01 no, T02 no), 11 (T01 no, T02 less), 20 (T01 no, T02 less), 21 (T01 no, T02 no), 29 (T02 less, T01 no), 30 (T01 less, T02 no), 32 (T01 no, T02 less), 33 (T01 less, T02 no), 40 (T02, T01 less), 41 (T01, T02 less), 42 (T01 less, T02 no), 43 (T01 no, T02 no), 44 (T02, T01 no), 45 (T01 no, T02 no), 46 (T01 no, T02 no)',
            'left': '0 (T13 less, T14 no), 1 (T13 no, T14 no), 2 (T13 no, T14 no), 4 (T13 no, T14 no), 5 (T13 no, T14 no), 6 (T13 less, T14 no), 7 (T13 no, T14 less), 8 (T13 no, T14 no), 10 (T13 no, T14 fix), 12 (T13 less, T14 no), 13 (T13 fix, T14 less), 15 (T13 no, T14 no), 16 (T13 fix, T14 fix), 18 (T13 no, T14 fix), 19 (T13 fix, T14 fix), 21 (T13 no, T14 no), 23 (T13 no, T14 no), 24 (T13 fix, T14 fix), 25 (T13 fix, T14 fix), 27 (T13 no, T14 less), 28 (T13, T14 no), 30 (T13 fix, T14 fix), 36 (T13 no, T14 less), 37 (T13 less, T14 fix), 40 (T13, T14 less)',
            'right': '0 (T05 no, T06 no), 1 (T05 less, T06 no), 2 (T05 no, T06 less), 3 (T05 no, T06 no), 7 (T05 no, T06 no), 9 (T05 no, T06 no), 11 (T05 no, T06 no), 13 (T05 no, T06 no), 14 (T05 no, T06 no), 17 (T05 no, T06 no), 22 (T05 no, T06 fix), 23 (T05 no, T06 no), 24 (T05 less, T06 fix), 26 (T05 fix, T06 no), 31 (T05 no, T06 less),',}
camera_sample_ratio = {'normal': 0.33, 'less': 0.15, 'fix': 10, 'no': 1}
save_csv = '/data/jupiter/datasets/dust_datasets/halo_dust_smudge_JUPD-1105_2024-10-23/selected_for_label.csv'
select_by_camera_sample_ratio(seq_dfs, selected, camera_sample_ratio, save_csv)

back 34 T10 less 185 28
back 34 T09 no 184 1
back 35 T09 less 187 28
back 35 T10 no 187 1
back 38 T10 normal 187 62
back 38 T09 no 188 1
back 39 T09 normal 18 6
back 39 T10 less 18 3
front 8 T01 less 102 15
front 8 T02 no 102 1
front 9 T02 less 102 15
front 9 T01 no 101 1
front 10 T01 no 186 1
front 10 T02 no 186 1
front 11 T01 no 99 1
front 11 T02 less 101 15
front 20 T01 no 101 1
front 20 T02 less 102 15
front 21 T01 no 101 1
front 21 T02 no 100 1
front 29 T02 less 187 28
front 29 T01 no 187 1
front 30 T01 less 99 15
front 30 T02 no 100 1
front 32 T01 no 118 1
front 32 T02 less 118 18
front 33 T01 less 187 28
front 33 T02 no 187 1
front 40 T02 normal 113 37
front 40 T01 less 113 17
front 41 T01 normal 112 37
front 41 T02 less 112 17
front 42 T01 less 111 17
front 42 T02 no 112 1
front 43 T01 no 204 1
front 43 T02 no 205 1
front 44 T02 normal 185 61
front 44 T01 no 186 1
front 45 T01 no 112 1
front 45 T02 no 112 1
front 46 T01 no 23 1
front 46 T02 no 23 1
left 0 T13 less 186 28
left 0

In [33]:
def parse_time_intervals(s):
    interval_strs = s.split(', ')
    intervals = []
    for interval_str in interval_strs:
        ts = interval_str.split(' - ')
        intervals.append(ts)
    return intervals

print(parse_time_intervals('2024-10-18T15:28:21.557000 - 2024-10-18T15:28.46.865000'))

[['2024-10-18T15:28:21.557000', '2024-10-18T15:28.46.865000']]


#### Select human in corn stubble and manure data

In [55]:
# for selection of images for labeling - halo_human_w_corn_stubble_0812_oct
selected = {'moving_random_sample': list(range(0, 12)) + list(range(48, 60)) + list(range(96, 108)) + list(range(120, 132)),
            'moving_random_sample_exclude_tps': list(range(60, 72)),
            'static_w_human_easy': [12, 15, 16, 17, 24, 32, 33, 34, 35, 39, 40, 41, 79],
            'static_w_human_medium': [20, 25, 84],
            'static_w_human_hard': [29, 80, 85],
            'static_w_human_short': [14, 22, 23, 27, 36, 37, 38, 72, 73, 74, 75, 76, 77, 78, 86],
            'static_wo_human': [13, 18, 19, 21, 26, 28, 30, 31, 42, 43, 44, 45, 46, 47, 81, 82, 83] + list(range(87, 96)) + list(range(108, 120))}
hard_selected = {29: ['2024-10-17T19:34:53.62700', '2024-10-17T19:35:22.265000'], 
                 80: ['2024-10-17T23:30:00.839000', '2024-10-17T23:30:30.143000'], 
                 85: ['2024-10-18T15:28:21.557000', '2024-10-18T15:28:46.865000']}
sample_rates = {'moving_random_sample': 0.05, 'moving_random_sample_exclude_tps': 0.05, 
               'static_w_human_easy': 0.05, 'static_w_human_medium': 0.1, 'static_w_human_hard': 0.5, 
               'static_w_human_short': 2, 'static_wo_human': 1}
selected_ids = []
for i, seq_df in enumerate(seq_dfs):
    # get collection category
    collection = ''
    for _collection, i_list in selected.items():
        if i in i_list:
            collection = _collection
    # get sampling type
    sample_rate = sample_rates[collection]
    if 'moving' in collection:
        # sample on all images except FPs (actually TPs)
        if 'exclude_tps' in collection:
            pred_seq_df = pred_df[pred_df['unique_id'].isin(seq_df['unique_id'])]
            sub_df = pred_seq_df[pred_seq_df.state != 'false_positive']
        # sample on all images
        else:
            sub_df = seq_df
    else:
        # sample on human images (FPs)
        if collection == 'static_w_human_easy' or collection == 'static_w_human_medium':
            pred_seq_df = pred_df[pred_df[id].isin(seq_df[id])]
            sub_df = pred_seq_df[pred_seq_df.state == 'false_positive']
        # sample on selected time window (with hard humans)
        elif collection == 'static_w_human_hard':
            assert i in hard_selected
            t1, t2 = hard_selected[i]
            sub_df = seq_df[(seq_df.collected_on >= t1) & (seq_df.collected_on <= t2)]
        # sample on all images but with very low sample rate
        else:
            sub_df = seq_df
    # do sampling
    if sample_rate < 1:
        sample_df = sub_df.sample(frac=sample_rate)
    else:
        sample_df = sub_df.sample(n=min(sample_rate, len(sub_df)))
    selected_ids += sample_df.id.to_list()
    print(i, len(seq_df), len(sub_df), len(sample_df), collection)
selected_ids = list(set(selected_ids))
selected_df = pd.DataFrame(data={'id': selected_ids})
print(selected_df.shape)
selected_df.to_csv('/data3/jupiter/datasets/model_positive/halo_human_w_corn_stubble_0812_oct/selected_for_label.csv', index=False)

0 516 516 26 moving_random_sample
1 520 520 26 moving_random_sample
2 520 520 26 moving_random_sample
3 505 505 25 moving_random_sample
4 509 509 25 moving_random_sample
5 509 509 25 moving_random_sample
6 505 505 25 moving_random_sample
7 502 502 25 moving_random_sample
8 502 502 25 moving_random_sample
9 506 506 25 moving_random_sample
10 507 507 25 moving_random_sample
11 507 507 25 moving_random_sample
12 250 44 2 static_w_human_easy
13 251 251 1 static_wo_human
14 251 251 2 static_w_human_short
15 257 55 3 static_w_human_easy
16 250 52 3 static_w_human_easy
17 250 15 1 static_w_human_easy
18 250 250 1 static_wo_human
19 250 250 1 static_wo_human
20 250 56 6 static_w_human_medium
21 255 255 1 static_wo_human
22 261 261 2 static_w_human_short
23 261 261 2 static_w_human_short
24 420 33 2 static_w_human_easy
25 424 82 8 static_w_human_medium
26 423 423 1 static_wo_human
27 438 438 2 static_w_human_short
28 436 436 1 static_wo_human
29 436 30 15 static_w_human_hard
30 424 424 1 static

In [30]:
# for selection of images for labeling - halo_manure_data_candidate
selected = {'front': 
            {0: [['2024-10-22T22:03:58.427000', '2024-10-22T22:04:12.080000'], ['2024-10-22T22:05:23.009000', '2024-10-22T22:05:34.664000'], ['2024-10-22T22:06:29.942000', '2024-10-22T22:06:45.926000']], 
             1: [['2024-10-22T22:10:23.708000', '2024-10-22T22:10:36.695000'], ['2024-10-22T22:11:15.656000', '2024-10-22T22:11:21.650000']], 
             2: [['2024-10-22T22:19:16.841000', '2024-10-22T22:19:24.167000'], ['2024-10-22T22:21:23.048000', '2024-10-22T22:21:31.706000'], ['2024-10-22T22:24:19.538000', '2024-10-22T22:24:35.522000'], ['2024-10-22T22:30:03.527000', '2024-10-22T22:30:12.851000']], 
             4: [['begin', '2024-10-23T19:03:09.154000'], ['2024-10-23T19:08:41.155000', '2024-10-23T19:09:17.119000']]},
            'right': {0: [['2024-10-22T21:59:24.035000', '2024-10-22T21:59:27.365000'], ['2024-10-22T22:04:10.415000', '2024-10-22T22:04:16.076000']], 
                      2: [['2024-10-22T22:20:55.409000', '2024-10-22T22:20:58.739000'], ['2024-10-22T22:21:08.396000', '2024-10-22T22:21:18.053000']]},
            'back': {0: [['2024-10-22T22:03:05.480000', '2024-10-22T22:03:20.132000'], ['2024-10-22T22:03:54.764000', '2024-10-22T22:04:10.082000'], ['2024-10-22T22:04:15.743000', '2024-10-22T22:04:23.069000'], ['2024-10-22T22:04:27.731000', '2024-10-22T22:04:40.385000']], 
                     1: [['2024-10-22T22:11:57.614000', '2024-10-22T22:12:02.276000']], 
                     2: [['2024-10-22T22:20:07.457000', '2024-10-22T22:20:16.448000'], ['2024-10-22T22:21:12.393000', '2024-10-22T22:21:43.694000'], ['2024-10-22T22:28:29.289000', '2024-10-22T22:28:40.610000']], 
                     4: [['begin', '2024-10-23T19:03:09.154000'], ['2024-10-23T19:03:22.141000', '2024-10-23T19:03:29.134000'], ['2024-10-23T19:03:50.779000', '2024-10-23T19:04:06.097000'], ['2024-10-23T19:06:03.646000', '2024-10-23T19:06:27.622000']]},
            'left': {0: [['2024-10-22T22:02:57.155000', '2024-10-22T22:03:04.814000'], ['2024-10-22T22:04:20.738000', '2024-10-22T22:04:33.392000']], 
                     2: [['2024-10-22T22:19:22.836000', '2024-10-22T22:19:31.161000'], ['2024-10-22T22:21:01.737000', '2024-10-22T22:21:28.377000'], ['2024-10-22T22:22:43.634000', '2024-10-22T22:22:55.623000'], ['2024-10-22T22:28:25.293000', '2024-10-22T22:28:30.621000']], 
                     4: [['2024-10-23T19:05:59.983000', '2024-10-23T19:06:17.632000'], ['2024-10-23T19:07:52.870000', '2024-10-23T19:08:01.861000'], ['2024-10-23T19:09:15.787000', '2024-10-23T19:09:25.444000']]}}
all_left_cameras = {'front': ['T01', 'T02'], 'right': ['T05', 'T06'], 'back': ['T09', 'T10'], 'left': ['T13', 'T14']}
selected_ids = []
for pod, cameras in all_left_cameras.items():
    selected_times = selected[pod]
    for seq_i, t1t2_list in selected_times.items():
        for t1t2 in t1t2_list:
            seq_df = seq_dfs[seq_i]
            seq_df = seq_df[seq_df.camera_location.isin(cameras)]
            if t1t2 == 'all':
                t1, t2 = seq_df.iloc[0].collected_on, seq_df.iloc[-1].collected_on
            else:
                t1, t2 = t1t2
                if t1 == 'begin':
                    t1 = seq_df.iloc[0].collected_on
                if t2 == 'end':
                    t2 = seq_df.iloc[-1].collected_on
            selected_df = seq_df[(seq_df.collected_on >= t1) & (seq_df.collected_on <= t2)]
            selected_ids += selected_df.id.to_list()
            print(pod, seq_i, t1, t2, len(seq_df), len(selected_df))
selected_ids = list(set(selected_ids))
selected_df = pd.DataFrame(data={'id': selected_ids})
print(selected_df.shape)
selected_df.to_csv('/data3/jupiter/datasets/model_positive/halo_manure_data_candidate/selected_for_label.csv', index=False)

front 0 2024-10-22T22:03:58.427000 2024-10-22T22:04:12.080000 1739 65
front 0 2024-10-22T22:05:23.009000 2024-10-22T22:05:34.664000 1739 63
front 0 2024-10-22T22:06:29.942000 2024-10-22T22:06:45.926000 1739 75
front 1 2024-10-22T22:10:23.708000 2024-10-22T22:10:36.695000 726 43
front 1 2024-10-22T22:11:15.656000 2024-10-22T22:11:21.650000 726 38
front 2 2024-10-22T22:19:16.841000 2024-10-22T22:19:24.167000 3037 23
front 2 2024-10-22T22:21:23.048000 2024-10-22T22:21:31.706000 3037 24
front 2 2024-10-22T22:24:19.538000 2024-10-22T22:24:35.522000 3037 48
front 2 2024-10-22T22:30:03.527000 2024-10-22T22:30:12.851000 3037 51
front 4 2024-10-23T19:02:59.497000 2024-10-23T19:03:09.154000 1597 53
front 4 2024-10-23T19:08:41.155000 2024-10-23T19:09:17.119000 1597 112
right 0 2024-10-22T21:59:24.035000 2024-10-22T21:59:27.365000 1749 7
right 0 2024-10-22T22:04:10.415000 2024-10-22T22:04:16.076000 1749 36
right 2 2024-10-22T22:20:55.409000 2024-10-22T22:20:58.739000 3031 21
right 2 2024-10-22T22:

#### Select failure case data

In [7]:
# # for selection of images for binary labeling - halo_failure_case_of_box_in_dust
# selected = [3, 5, [9, 1/5], [10, 1/5], [11, 1/6], 
#     [12, [['2024-04-23T18:11:28', '2024-04-23T18:18:12'], ['2024-04-23T18:27:40', '2024-04-23T18:28:38'], ['2024-04-23T18:29:05', '2024-04-23T18:29:57'], 
#         ['2024-04-23T18:48:44', '2024-04-23T18:50:59'], ['2024-04-23T19:01:50', '2024-04-23T19:03:37']]], 
#     [13, [['2024-04-23T19:47:48', '2024-04-23T19:47:59']]], 
#     [14, [['2024-04-23T20:12:40', '2024-04-23T20:13:02'], ['2024-04-23T20:15:53', '2024-04-23T20:17:49'], ['2024-04-23T20:22:19', '2024-04-23T20:22:45'], 
#         ['2024-04-23T20:32:47', '2024-04-23T20:33:18'], ['2024-04-23T20:34:48', '2024-04-23T20:35:56'], ['2024-04-23T20:38:00', '2024-04-23T20:38:17'], 
#         ['2024-04-23T20:39:55', '2024-04-23T20:41:31'], ['2024-04-23T20:42:57', '2024-04-23T20:48:27']]], 
#     [16, 1/2]]
# selected_ids = []
# for s in selected:
#     if isinstance(s, int):
#         selected_ids += seq_dfs[s].id.to_list()
#     else:
#         si, sj = s
#         if isinstance(sj, float):
#             selected_ids += seq_dfs[si].iloc[:int(len(seq_dfs[si])*sj)].id.to_list()
#         else:
#             for t1, t2 in sj:
#                 selected_ids += seq_dfs[si][(seq_dfs[si].collected_on >= t1) & (seq_dfs[si].collected_on <= t2)].id.to_list()
#     # print(s, len(selected_ids))
# selected_ids = list(set(selected_ids))
# selected_df = pd.DataFrame(data={'id': selected_ids})
# print(selected_df.shape)
# selected_df.to_csv('/data/jupiter/datasets/halo_failure_case_of_box_in_dust/selected_for_label.csv', index=False)

#### Select human in dust test images for labeling

In [7]:
def select_ids_for_label(seq_dfs, all_cameras, selected, save_csv):
    selected_ids = []
    for pod, cameras in all_cameras.items():
        print(pod, cameras)
        for i,seq_df in enumerate(seq_dfs):
            if i in selected[pod]:
                sub_df = seq_df[seq_df.camera_location.isin(cameras)]
                selected_ids += sub_df.id.to_list()
    selected_ids = list(set(selected_ids))
    selected_df = pd.DataFrame(data={'id': selected_ids})
    print(selected_df.shape)
    if save_csv:
        selected_df.to_csv(save_csv, index=False)
        return None
    return selected_df

def recover_skipped_human_label(data_dir, seq_dfs, suffix, same_human_sequence):
    all_cameras = {'front': ['T01', 'T02', 'T03', 'T04'], 'right': ['T05', 'T06', 'T07', 'T08'], 'back': ['T09', 'T10', 'T11', 'T12'], 'left': ['T13', 'T14', 'T15', 'T16']}

    raw_ms_df = pd.read_csv(os.path.join(data_dir, 'master_annotations.csv'))
    raw_ms_df['camera_pair'] = raw_ms_df['unique_id'].apply(lambda s: s[-7:])
    raw_ms_df['rectified_label_save_path'] = ''
    raw_ms_df['use_recovered_label_in_sequence'] = True
    labeled_ms_df = pd.read_csv(os.path.join(data_dir+suffix, 'master_annotations.csv'))
    labeled_ms_df.drop(columns=["label_counts"], inplace=True)
    labeled_ms_df['camera_pair'] = labeled_ms_df['unique_id'].apply(lambda s: s[-7:])
    print(raw_ms_df.shape, labeled_ms_df.shape)

    recovered_dfs = []
    for pod, seq_ids in same_human_sequence.items():
        for seq_id in seq_ids:
            # get seq df in pod
            seq_df = seq_dfs[seq_id]
            seq_df = seq_df[seq_df.camera_location.isin(all_cameras[pod])]
            # get corresponding raw seq df and labeled seq df
            raw_seq_df = raw_ms_df[raw_ms_df.id.isin(seq_df.id)]
            labeled_seq_df = labeled_ms_df[labeled_ms_df.id.isin(seq_df.id)]
            labeled_seq_df = labeled_seq_df.sort_values('collected_on')
            # get camera locations where there are human labels
            labeled_camera_pairs = labeled_seq_df.camera_pair.unique()
            for camera_pair in labeled_camera_pairs:
                raw_seq_cp_df = raw_seq_df[raw_seq_df.camera_pair == camera_pair]
                labeled_seq_cp_df = labeled_seq_df[labeled_seq_df.camera_pair == camera_pair]
                # assign label path to raw df
                for i, row in raw_seq_cp_df.iterrows():
                    labeled_rows = labeled_seq_cp_df[labeled_seq_cp_df.unique_id == row.unique_id]
                    if len(labeled_rows) > 0:
                        raw_seq_cp_df.loc[i, 'rectified_label_save_path'] = labeled_rows.iloc[0].rectified_label_save_path
                        raw_seq_cp_df.loc[i, 'use_recovered_label_in_sequence'] = False
                    else:
                        raw_seq_cp_df.loc[i, 'rectified_label_save_path'] = labeled_seq_cp_df.iloc[0].rectified_label_save_path
                recovered_dfs.append(raw_seq_cp_df)
            print(pod, seq_id, len(seq_df), len(raw_seq_df), len(labeled_seq_df), raw_seq_df.camera_pair.unique(), labeled_seq_df.camera_pair.unique())
    recovered_df = pd.concat(recovered_dfs, ignore_index=True)
    print(recovered_df.shape, labeled_ms_df.shape)

    # remove duplicates and add in extra sequence images from labeled_ms_df
    recovered_df = pd.concat([recovered_df.drop_duplicates(subset='unique_id'), labeled_ms_df[~labeled_ms_df.unique_id.isin(recovered_df.unique_id)]], ignore_index=True)
    # change relative path to label path
    recovered_df['rectified_label_save_path'] = recovered_df['rectified_label_save_path'].apply(lambda p: f'../{os.path.basename(data_dir)+suffix}/{p}')
    recovered_df['label_map'] = labeled_ms_df.iloc[0].label_map
    print(recovered_df.shape, labeled_ms_df.shape)
    recovered_df.to_csv(os.path.join(data_dir, 'label_recovered_master_annotations.csv'), index=False)

In [8]:
# # for selection of images for partial human labeling - halo_human_in_dust_day_collection_may29
# selected = {'front': {0: 'all', 1: ['begin', '2024-05-29T19:48:13'], 2: ['2024-05-29T20:28:50', 'end']},
#             'right': {2: ['begin', '2024-05-29T20:28:53']},
#             'back': {1: ['2024-05-29T20:02:08', 'end']},
#             'left': {1: ['2024-05-29T19:48:10', '2024-05-29T20:02:08']}}
# all_cameras = {'front': ['T01', 'T02', 'T03', 'T04'], 'right': ['T05', 'T06', 'T07', 'T08'], 'back': ['T09', 'T10', 'T11', 'T12'], 'left': ['T13', 'T14', 'T15', 'T16']}
# left_cameras = ['front-center-left', 'front-left-left', 'front-right-left', 'side-left-left', 'side-right-left', 'rear-left', 'T01', 'T02', 'T05', 'T06', 'T09', 'T10', 'T13', 'T14', 'I01', 'I02']

# selected_ids = []
# for pod, cameras in all_cameras.items():
#     cameras = set(cameras).intersection(left_cameras)
#     selected_times = selected[pod]
#     for seq_i, t1t2 in selected_times.items():
#         seq_df = seq_dfs[seq_i]
#         seq_df = seq_df[seq_df.camera_location.isin(cameras)]
#         if t1t2 == 'all':
#             t1, t2 = seq_df.iloc[0].collected_on, seq_df.iloc[-1].collected_on
#         else:
#             t1, t2 = t1t2
#             if t1 == 'begin':
#                 t1 = seq_df.iloc[0].collected_on
#             if t2 == 'end':
#                 t2 = seq_df.iloc[-1].collected_on
#         print(pod, seq_i, t1, t2, len(seq_df))
#         selected_ids += seq_df[(seq_df.collected_on >= t1) & (seq_df.collected_on <= t2)].id.to_list()
# selected_ids = list(set(selected_ids))
# selected_df = pd.DataFrame(data={'id': selected_ids})
# print(selected_df.shape)
# selected_df.to_csv('/data/jupiter/datasets/halo_human_in_dust_day_collection_may29/selected_for_label.csv', index=False)

# # recover skipped human labels in heavy dust, by images in the same sequence
# same_human_sequence = {'front': [0, 1, 3, 14], 'right': [11, 12, 13], 'back': [7, 8, 9, 10], 'left': [4, 5, 6]}
# suffix = '_human_labeled_stereo'
# recover_skipped_human_label(data_dir, seq_dfs, suffix, same_human_sequence)

# select images with dust on lens
left_cameras = {'front': ['T01', 'T02'], 'right': ['T05', 'T06'], 'back': ['T09', 'T10'], 'left': ['T13', 'T14']}
# selected = {'front': [1, 3, 5, 11, 14, 15], 'right': [15,], 'back': [], 'left': [6, 10]}  # all
# selected = {'front': [1, 3, 14], 'right': [], 'back': [], 'left': [6]}  # with human
selected = {'front': [5, 11], 'right': [15,], 'back': [], 'left': [10]}  # without human
save_csv = '/data/jupiter/datasets/halo_human_in_dust_day_collection_may29/dust_on_lens_without_human.csv'
select_ids_for_label(seq_dfs, left_cameras, selected, save_csv)

front ['T01', 'T02']
right ['T05', 'T06']
back ['T09', 'T10']
left ['T13', 'T14']
(1048, 1)


In [18]:
# # load in labeled dataset
# labeled_csv = '/data2/jupiter/datasets/halo_vehicles_driving_through_dust_images_nodust_reserved_labeled_maxfov_alleysson_depth0423/annotations.csv'
# labeled_df = pd.read_csv(labeled_csv)
# print(labeled_df.shape)
# labeled_ids = set(labeled_df.id.to_list())
# print(len(labeled_ids))

# # for selection of images for missing partial vehicle labeling - halo_vehicles_in_dust_collection_march2024
# left_cameras = {'left_pass': ['T09', 'T14', 'T13', 'T02'], 'right_pass': ['T10', 'T05', 'T06', 'T01']}
# selected = {'left_pass': [2, 3, 4, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 27, 28, 30, 31, 32, 33, 34, 35, 36], 
#             'right_pass': [3, 4, 5, 6, 7, 9, 10, 12, 13, 14, 15, 16, 17, 18, 20, 21, 26, 27, 37]}
# selected_df = select_ids_for_label(seq_dfs, left_cameras, selected, save_csv='')
# print(selected_df.shape)
# selected_df = selected_df[~selected_df.id.isin(labeled_ids)]
# print(selected_df.shape)
# selected_df.to_csv('/data/jupiter/datasets/halo_vehicles_in_dust_collection_march2024/selected_for_missing_label.csv', index=False)

left_pass ['T09', 'T14', 'T13', 'T02']
right_pass ['T10', 'T05', 'T06', 'T01']
(13141, 1)
(9893, 1)


In [11]:
# # for selection of images for missing partial human labeling - halo_human_in_dust_night_collection_june03
# left_cameras = {'front': ['T01', 'T02'], 'right': ['T05', 'T06'], 'back': ['T09', 'T10'], 'left': ['T13', 'T14']}
# all_cameras = {'front': ['T01', 'T02', 'T03', 'T04'], 'right': ['T05', 'T06', 'T07', 'T08'], 'back': ['T09', 'T10', 'T11', 'T12'], 'left': ['T13', 'T14', 'T15', 'T16']}
# selected = {'front': [25, 26, 27, 30]}
# select_ids_for_label(seq_dfs, left_cameras, selected, save_csv='')

# # for selection of images for missing partial human labeling - halo_human_in_dust_night_collection_june03_2
# left_cameras = {'front': ['T01', 'T02'], 'right': ['T05', 'T06'], 'back': ['T09', 'T10'], 'left': ['T13', 'T14']}
# selected = {'front': [0, 1, 2, 3, 15, 16, 17], 'right': [18,], 'back': [7, 8, 9, 10, 11], 'left': [4, 5, 6, 12, 13, 14]}
# save_csv = '/data/jupiter/datasets/halo_human_in_dust_night_collection_june03_2/selected_for_label.csv'
# select_ids_for_label(seq_dfs, left_cameras, selected, save_csv)

# # recover skipped human labels in heavy dust, by images in the same sequence
# same_human_sequence = {'front': [15, 17], 'right': [18], 'back': [7, 8, 9, 10], 'left': [5, 6]}
# suffix = '_human_labeled_stereo'
# recover_skipped_human_label(data_dir, seq_dfs, suffix, same_human_sequence)

# select images with dust on lens
left_cameras = {'front': ['T01', 'T02'], 'right': ['T05', 'T06'], 'back': ['T09', 'T10'], 'left': ['T13', 'T14']}
# selected = {'front': [], 'right': [], 'back': [], 'left': [12, 13, 14]}  # with human
selected = {'front': [16], 'right': [], 'back': [], 'left': []}  # without human
save_csv = '/data/jupiter/datasets/halo_human_in_dust_night_collection_june03_2/dust_on_lens_without_human.csv'
select_ids_for_label(seq_dfs, left_cameras, selected, save_csv)

front ['T01', 'T02']
right ['T05', 'T06']
back ['T09', 'T10']
left ['T13', 'T14']
(674, 1)


In [64]:
# for selection of images for missing partial human labeling - halo_human_in_dust_day_collection_back_june05
# recover skipped human labels in heavy dust, by images in the same sequence
same_human_sequence = {'back': [0, 1, 2, 3, 5, 6, 7]}
suffix = '_human_labeled_stereo'
recover_skipped_human_label(data_dir, seq_dfs, suffix, same_human_sequence)

<ipython-input-64-9e5b735fb503>:5: DtypeWarning: Columns (25) have mixed types.Specify dtype option on import or set low_memory=False.
  recovered_df, labeled_ms_df = recover_skipped_human_label(data_dir, seq_dfs, suffix, same_human_sequence)
/home/li.yu/anaconda3/envs/pytorchlightning/lib/python3.8/site-packages/pandas/core/indexing.py:1817: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_column(loc, value, pi)


(8740, 153) (3511, 277)
back 0 752 562 184 ['T09_T11' 'T10_T12' 'T10_T11'] ['T09_T11']
back 1 1216 911 114 ['T09_T11' 'T10_T12' 'T10_T11'] ['T09_T11']
back 2 248 186 11 ['T09_T11' 'T10_T12' 'T10_T11'] ['T09_T11']
back 3 1476 1102 196 ['T09_T11' 'T10_T12' 'T10_T11'] ['T09_T11']
back 5 1688 1262 576 ['T09_T11' 'T10_T12' 'T10_T11'] ['T10_T12' 'T10_T11']
back 6 2176 1631 967 ['T09_T11' 'T10_T12' 'T10_T11'] ['T10_T11' 'T10_T12']
back 7 1380 1034 687 ['T09_T11' 'T10_T12' 'T10_T11'] ['T10_T12' 'T10_T11']
(3538, 153) (3511, 277)
(4314, 278) (3511, 277)


In [8]:
# # for selection of images for missing partial human labeling - halo_human_in_dust_dusk_collection_front_june07
# left_cameras = {'front': ['T01', 'T02'], 'right': ['T05', 'T06'], 'back': ['T09', 'T10'], 'left': ['T13', 'T14']}
# selected = {'front': [5], 'right': [], 'back': [], 'left': []}
# save_csv = '/data/jupiter/datasets/halo_human_in_dust_dusk_collection_front_june07/selected_for_label.csv'
# select_ids_for_label(seq_dfs, left_cameras, selected, save_csv)

front ['T01', 'T02']
right ['T05', 'T06']
back ['T09', 'T10']
left ['T13', 'T14']
(3354, 1)


In [14]:
# # for selection of images for missing partial vehicle labeling - halo_vehicles_in_dust_collection_june06
# left_cameras = {'left_pass': ['T09', 'T14', 'T13', 'T02'], 'right_pass': ['T10', 'T05', 'T06', 'T01']}
# selected = {'left_pass': [11, 12, 15], 
#             'right_pass': [10, 13, 14]}
# save_csv = '/data/jupiter/datasets/halo_vehicles_in_dust_collection_june06/selected_for_label.csv'
# select_ids_for_label(seq_dfs, left_cameras, selected, save_csv)

left_pass ['T09', 'T14', 'T13', 'T02']
right_pass ['T10', 'T05', 'T06', 'T01']
(4513, 1)


In [8]:
# # for selection of images for missing partial vehicle labeling - halo_vehicles_in_dust_collection_june07
# left_cameras = {'left_pass': ['T09', 'T14', 'T13', 'T02'], 'right_pass': ['T10', 'T05', 'T06', 'T01']}
# selected = {'left_pass': [2, 6, 7, 10], 
#             'right_pass': [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12]}
# save_csv = '/data/jupiter/datasets/halo_vehicles_in_dust_collection_june07/selected_for_label.csv'
# select_ids_for_label(seq_dfs, left_cameras, selected, save_csv)

left_pass ['T09', 'T14', 'T13', 'T02']
right_pass ['T10', 'T05', 'T06', 'T01']
(14354, 1)


In [9]:
# # for selection of images for missing partial vehicle labeling - halo_vehicles_in_dust_collection_june04
# left_cameras = {'left_pass': ['T09', 'T14', 'T13', 'T02'], 'right_pass': ['T10', 'T05', 'T06', 'T01']}
# selected = {'left_pass': [1, 2, 3, 4, 5, 6, 7, 8, 9], 
#             'right_pass': [9]}
# save_csv = '/data/jupiter/datasets/halo_vehicles_in_dust_collection_june04/selected_for_label.csv'
# select_ids_for_label(seq_dfs, left_cameras, selected, save_csv)

left_pass ['T09', 'T14', 'T13', 'T02']
right_pass ['T10', 'T05', 'T06', 'T01']
(26470, 1)


In [9]:
def select_ids_for_label_v2(seq_dfs, selected, save_csv):
    selected_ids = []
    for i,seq_df in enumerate(seq_dfs):
        for camera, slices in selected.items():
            cam_df = seq_df[seq_df.camera_location == camera]
            for slice in slices:
                if isinstance(slice, list):
                    if slice[0] != i:
                        continue
                    cam_df = cam_df.sort_values('collected_on')
                    begin = end = ''
                    if slice[1] == 'begin':
                        begin = cam_df.iloc[0].collected_on
                    elif slice[1] == 'middle':
                        begin = cam_df.iloc[len(cam_df)//2].collected_on
                    else:
                        begin = slice[1]
                    if slice[2] == 'middle':
                        end = cam_df.iloc[len(cam_df)//2].collected_on
                    elif slice[2] == 'end':
                        end = cam_df.iloc[-1].collected_on
                    else:
                        end = slice[2]
                    sub_df = cam_df[(cam_df.collected_on >= begin) & (cam_df.collected_on <= end)]
                    selected_ids += sub_df.id.to_list()
                    print(camera, i, len(sub_df))
                else:
                    if slice != i:
                        continue
                    selected_ids += cam_df.id.to_list()
                    print(camera, i, len(cam_df))
    selected_ids = list(set(selected_ids))
    selected_df = pd.DataFrame(data={'id': selected_ids})
    print(selected_df.shape)
    if save_csv:
        selected_df.to_csv(save_csv, index=False)
        return None
    return selected_df

In [14]:
# # for selection of images for running PP and model inference - Jupiter_bedrock_40013_20240617_dust_sequences
# all_cameras = {'front': ['T01', 'T02', 'T03', 'T04'], 'right': ['T05', 'T06', 'T07', 'T08'], 'back': ['T09', 'T10', 'T11', 'T12'], 'left': ['T13', 'T14', 'T15', 'T16']}
# selected = {'front': [0, 1, 4, 8, 9], 'right': [], 'back': [], 'left': []}
# save_csv = '/data/jupiter/datasets/dust_datasets/Jupiter_bedrock_40013_20240617_dust_sequences/selected_for_pp.csv'
# select_ids_for_label(seq_dfs, all_cameras, selected, save_csv)
# selected_ids_df = pd.read_csv(save_csv)
# selected_ann_df = df[df.id.isin(selected_ids_df.id)]
# print(selected_ann_df.shape)
# selected_ann_df.to_csv('/data/jupiter/datasets/dust_datasets/Jupiter_bedrock_40013_20240617_dust_sequences/selected_for.csv', index=False)

# select dust on lens only (no dust in air) sequences:
selected_box = {'T02': [[1, '2024-06-17T21:24:51.901000', 'end'], 2, 3]}
selected_human = {'T01': [[10, 'begin', '2024-06-17T21:53:49:495000']], 
            'T02': [[4, 'begin', '2024-06-17T21:32:18.787000'], [9, 'middle', 'end'], [10, 'begin', '2024-06-17T21:53:49:495000']]}
selected_human_v2 = {'T02': [[4, '2024-06-17T21:28:44.335000', '2024-06-17T21:32:18.787000'], [9, '2024-06-17T21:45:47.644000', 'end']]}
selected_box_csv = os.path.join(data_dir, 'dust_on_lens_with_box.csv')
selected_human_csv = os.path.join(data_dir, 'dust_on_lens_with_human.csv')
selected_human_v2_csv = os.path.join(data_dir, 'dust_on_lens_with_human_v3.csv')
# select_ids_for_label_v2(seq_dfs, selected_box, selected_box_csv)
# select_ids_for_label_v2(seq_dfs, selected_human, selected_human_csv)
select_ids_for_label_v2(seq_dfs, selected_human_v2, selected_human_v2_csv)

# # select dust on lens and dust in air with manny sequences
# selected_human = {'T02': [[9, 'begin', 'middle']]}
# selected_human_csv = os.path.join(data_dir, 'mixed_dust_with_human.csv')
# select_ids_for_label_v2(seq_dfs, selected_human, selected_human_csv)

T02 4 645
T02 9 3
(648, 1)


In [31]:
# for selection and triaging of dust on lens IQ test set - halo_dust_on_lens_blur_dataset_v3_20240807
all_cameras = {'front': ['T01', 'T02', 'T03', 'T04'], 'right': ['T05', 'T06', 'T07', 'T08'], 'back': ['T09', 'T10', 'T11', 'T12'], 'left': ['T13', 'T14', 'T15', 'T16']}
dedup = {'front': {0:[1, 3], 1: [1, 3], 8: [1, 3, 4], 9: [1,2,3,4], 10: [1,2,3,4], 17:[1, 3], 18:[1, 3], 19:[1, 3], 20:[1, 3], 21:[1, 3], 22:[1, 3], 23:[1, 3], 
                   24:[1, 3], 25:[1, 3], 26:[1, 3], 32:[1, 3], 33:[1, 3], 36:[1,2,3,4], 37:[1,2,3,4], 38:[1,2,3,4], 39:[1,2,3,4], 40:[1,2,3,4], 41:[1,2], 43:[1,2]}, 
         'right': {2:[5,7], 3:[5,7], 4:[5,7], 11:[5,6,7,8], 12:[5,6,7,8], 28:[7], 29:[7], 36:[5,6,7,8], 37:[5,6,7,8], 38:[5,6,7,8], 39:[5,6,7,8], 40:[5,6,7,8]}, 
         'back': {13:[9,10,11,12], 14:[9,10,11,12], 36:[9,10,11,12], 37:[9,10,11,12], 38:[9,10,11,12], 39:[9,10,11,12], 40:[9,10,11,12]}, 
         'left': {5:[13,15], 8:[16], 9:[16], 10:[16], 11:[16], 12:[16], 13:[16], 14:[16], 15:[13,14,15,16], 16:[13,14,15,16], 27:[14,16], 36:[13,14,15,16], 37:[13,14,15,16], 
                  38:[13,14,15,16], 39:[13,14,15,16], 40:[13,14,15,16], 42:[13,14,15,16]}}
remove = {'front': [4, 6, 14, 15, 45], 'right': [6, 13, 14, 15, 16, 34, 44], 'back': [8, 11, 15, 16, 26, 27, 30, 31, 32, 33, 34, 35], 'left': [6, 7, 24, 28, 31]}
# load already triaged ids
triage_df = pd.read_csv(os.path.join(data_dir, 'to_triage.csv'))
print(triage_df.shape)
# dedup images from each sequence, such that there is at most 30 images per minute per camera
pods = ['front', 'left', 'right', 'back']
limit = 30
selected_dfs = []
for i, seq_df in enumerate(seq_dfs):
    for pod in pods:
        # check for remove list
        if i in remove[pod]:
            continue
        # dedup
        if i in dedup[pod]:
            cameras = [f'T{str(seq_i).zfill(2)}' for seq_i in dedup[pod][i]]
            for cam in cameras:
                cam_df = seq_df[seq_df.camera_location == cam]
                cam_df = cam_df[~cam_df.id.isin(triage_df.id)]
                if len(cam_df) == 0:
                    continue
                cam_df = cam_df.sort_values('collected_on')
                time_span = (cam_df.iloc[-1].datetime - cam_df.iloc[0].datetime).total_seconds()
                limit_ids = int(time_span / 60 * limit)
                selected_dfs.append(cam_df.sample(min(len(cam_df), limit_ids)))
                # print(i, pod, cam, time_span, len(cam_df), limit_ids)
        # else:
        #     pod_df = seq_df[seq_df.camera_location.isin(all_cameras[pod])]
        #     selected_dfs.append(pod_df)
        #     print(i, pod, cam, len(pod_df))
selected_df = pd.concat(selected_dfs, ignore_index=True)
print(df.shape, selected_df.shape)
# save ids to csv
selected_df[['id']].to_csv(os.path.join(data_dir, 'v4_dedup_triaged.csv'), index=False)

(1085, 1)
(19813, 149) (8534, 150)


### create video

In [6]:
def read_csvs(pred_root_dir, model, labeled_dataset):
    pred_df = pd.read_csv(os.path.join(pred_root_dir, model, labeled_dataset, 'output.csv'))
    if not 'state' in pred_df:
        pred_df['state'] = pred_df['result_state']
    if os.path.isfile(os.path.join(pred_root_dir, model, labeled_dataset, 'dust_ratio.csv')):
        dust_df = pd.read_csv(os.path.join(pred_root_dir, model, labeled_dataset, 'dust_ratio.csv'))
        print(labeled_dataset, pred_df.shape, dust_df.shape)
        df = pred_df[['unique_id', 'state', 'camera_location', 'operation_time']].merge(dust_df, on='unique_id')
    else:
        df = pred_df[['unique_id', 'id', 'state', 'camera_location', 'operation_time']]
    all_cameras = {'Front Pod': ['T01', 'T02', 'T03', 'T04'], 'Right Pod': ['T05', 'T06', 'T07', 'T08'], 'Rear Pod': ['T09', 'T10', 'T11', 'T12'], 'Left Pod': ['T13', 'T14', 'T15', 'T16']}
    all_cameras = {c: pod for pod, cameras in all_cameras.items() for c in cameras}
    df['pod'] = df.camera_location.apply(lambda c: all_cameras[c])
    return df

In [16]:
# load in model predictions
pred_root_dir = '/data/jupiter/li.yu/exps/driveable_terrain_model/'
# model = '9_2_unofficial_rev1_lying_down_sitting_25_ep_prod_weights_10_lo_10_tr_br_dr_p2_v_p1_m_p1_u_dust_0624'
# model = '9_2_unofficial_rev1_lying_down_sitting_25_ep_prod_weights_10_lo_5_tr_br_dr_p2_v_p1_m_p05_u_2_dust_0626'
model = '11_3_rev1_train_human_test_dean_multires_ignore_trees_1p25_u_p5_h_p2_v_ft_from_rev1_22kdust_ft_p15dust_h_fresh_dust11'
# unlabeled pred
# pred_df = read_csvs(pred_root_dir, model, os.path.basename(data_dir))
pred_df = read_csvs(pred_root_dir, model, 'dust_datasets/Jupiter_1RW9640DKPA085013_20241017_dust_stereo')
print(pred_df.shape)
# # labeled pred
# suffix = '_human_labeled_stereo'
# labeled_pred_df = read_csvs(pred_root_dir, model, os.path.basename(data_dir)+suffix)
# print(labeled_pred_df.shape)
# # put labeled rows to raw, unlabeled df
# pred_df.loc[pred_df.unique_id.isin(labeled_pred_df.unique_id), ['total_averaged_dust_conf', 'state']] = labeled_pred_df[['total_averaged_dust_conf', 'state']].values
pred_df.head(2)

dust_datasets/Jupiter_1RW9640DKPA085013_20241017_dust_stereo (12081, 24) (12081, 7)
(12081, 11)


,unique_id,state,camera_location,operation_time,id,gt_dust_ratio,total_averaged_dust_conf,total_thresholded_dust_ratio,masked_avg_dust_conf,masked_dust_ratio,pod
0,671a9536592c3e749eb2bef4_T10_T11,true_negative,T10,daytime,671a9536592c3e749eb2bef4,0.0,0.000013,0.0,0.000007,0.0,Rear Pod
1,671a96abb56c97a09c4821fb_T10_T11,true_negative,T10,daytime,671a96abb56c97a09c4821fb,0.0,0.000011,0.0,0.000005,0.0,Rear Pod


In [18]:
# read from multiple cameras and put in once frame
# left_pass_pairs = ['T09_T11', 'T14_T16', 'T14_T15', 'T13_T15']
# right_pass_pairs = ['T05_T07', 'T10_T12', 'T06_T08', 'T06_T07']
# cameras = [f'T{str(i+1).zfill(2)}' for i in range(16)][12:]
all_cameras = {'front': ['T01', 'T02', 'T03', 'T04'], 'right': ['T05', 'T06', 'T07', 'T08'], 'back': ['T09', 'T10', 'T11', 'T12'], 'left': ['T13', 'T14', 'T15', 'T16']}
# all_left_cameras = {'front': ['T01', 'T02'], 'right': ['T05', 'T06'], 'back': ['T09', 'T10'], 'left': ['T13', 'T14']}
# plain_left_cameras = {'left8': ['T01', 'T02', 'T05', 'T06', 'T09', 'T10', 'T13', 'T14']}
# left_cameras = {'left_pass': ['T09', 'T14', 'T13', 'T02'], 'right_pass': ['T10', 'T05', 'T06', 'T01']}
for pod, cameras in all_cameras.items():
    print(pod, cameras)
    H = 2  # number of camera rows
    W = 4  # number of camera cols
    for i,seq_df in enumerate(seq_dfs):
        # # select sequence to create video
        # if not i in selected[pod]:
        #     continue
        # check model prediction and filter by dust/seg outputs
        pred_seq_df = pred_df[pred_df.id.isin(seq_df.id)]
        labeled_ids = pred_seq_df.id.to_list()

        cam_dfs = [seq_df[seq_df.camera_location == c] for c in cameras]
        print(i, [len(cdf) for cdf in cam_dfs])
        min_len = min(len(cdf) for cdf in cam_dfs)
        cam_dfs = [cdf.sort_values('collected_on').iloc[:min_len] for cdf in cam_dfs]
        if min_len < 2:
            continue
        # print(i, [cdf.iloc[0].collected_on for cdf in cam_dfs])

        frame = read_raw_image_by_row(data_dir, seq_df.iloc[0])
        height, width, layers = frame.shape
        # print(height, width, layers)

        # .avi MJPG,  .mp4 MP4V
        video_dir = os.path.join(data_dir, f'videos_{len(cameras)}cams_pred')
        os.makedirs(video_dir, exist_ok=True)
        video_name = os.path.join(video_dir, f'{pod}_seq{str(i).zfill(2)}.mp4')
        video = cv2.VideoWriter(video_name, cv2.VideoWriter_fourcc(*'MP4V'), 6, (width*W,height*H), isColor=True)

        for ii in tqdm(range(min_len)):
            try:
                canvas = np.zeros((height*H, width*W, 3), dtype=np.uint8)
                for ci in range(len(cam_dfs)):
                    cam_df_row = cam_dfs[ci].iloc[ii]
                    frame = read_raw_image_by_row(data_dir, cam_df_row)
                    c = (255,0,0)
                    s = f'{cam_df_row.camera_location} {cam_df_row.collected_on}'
                    if cam_df_row.id in labeled_ids:
                        pred_row = pred_seq_df[pred_seq_df.id == cam_df_row.id].iloc[0]
                        s += f' pred: {pred_row.state}, dust: {pred_row.total_averaged_dust_conf:.4f}'
                        if pred_row.state == 'true_positive' or pred_row.state == 'false_positive':
                            c = (0,0,255)
                    frame = cv2.putText(frame, s, 
                            (40,50), cv2.FONT_HERSHEY_SIMPLEX, 1, c, 2, cv2.LINE_AA)
                    fi, fj = ci // W, ci % W
                    canvas[height*fi:height*(fi+1), width*fj:width*(fj+1)] = frame
                video.write(canvas)
            except:
                print(f'{ii}th image read failed')

        # cv2.destroyAllWindows()
        video.release()
        # break

front ['T01', 'T02', 'T03', 'T04']
0 [0, 0, 0, 0]
1 [0, 0, 0, 0]
2 [0, 0, 0, 0]
3 [0, 0, 0, 0]
4 [0, 0, 0, 0]
5 [0, 0, 0, 0]
6 [0, 0, 0, 0]
7 [0, 0, 0, 0]
8 [0, 0, 0, 0]
9 [0, 0, 0, 0]
10 [0, 0, 0, 0]
11 [0, 0, 0, 0]
12 [0, 0, 0, 0]
13 [0, 0, 0, 0]
14 [0, 0, 0, 0]
15 [0, 0, 0, 0]
16 [0, 0, 0, 0]
17 [0, 0, 0, 0]
18 [0, 0, 0, 0]
19 [0, 0, 0, 0]
20 [0, 0, 0, 0]
21 [0, 0, 0, 0]
22 [0, 0, 0, 0]
23 [0, 0, 0, 0]
24 [0, 0, 0, 0]
25 [0, 0, 0, 0]
26 [0, 0, 0, 0]
27 [0, 0, 0, 0]
28 [0, 0, 0, 0]
29 [0, 0, 0, 0]
30 [0, 0, 0, 0]
31 [0, 0, 0, 0]
32 [0, 0, 0, 0]
33 [0, 0, 0, 0]
34 [0, 0, 0, 0]
35 [0, 0, 0, 0]
36 [0, 0, 0, 0]
37 [0, 0, 0, 0]
38 [0, 0, 0, 0]
39 [0, 0, 0, 0]
40 [0, 0, 0, 0]
41 [0, 0, 0, 0]
42 [0, 0, 0, 0]
43 [0, 0, 0, 0]
44 [0, 0, 0, 0]
45 [0, 0, 0, 0]
46 [0, 0, 0, 0]
47 [0, 0, 0, 0]
48 [0, 0, 0, 0]
49 [0, 0, 0, 0]
50 [0, 0, 0, 0]
51 [0, 0, 0, 0]
right ['T05', 'T06', 'T07', 'T08']
0 [0, 0, 0, 0]
1 [0, 0, 0, 0]
2 [0, 0, 0, 0]
3 [0, 0, 0, 0]
4 [0, 0, 0, 0]
5 [0, 0, 0, 0]
6 [0, 0, 0, 0]
7 [

OpenCV: FFMPEG: tag 0x5634504d/'MP4V' is not supported with codec id 12 and format 'mp4 / MP4 (MPEG-4 Part 14)'
OpenCV: FFMPEG: fallback to use tag 0x7634706d/'mp4v'
100%|██████████| 33/33 [00:12<00:00,  2.63it/s]
OpenCV: FFMPEG: tag 0x5634504d/'MP4V' is not supported with codec id 12 and format 'mp4 / MP4 (MPEG-4 Part 14)'
OpenCV: FFMPEG: fallback to use tag 0x7634706d/'mp4v'


1 [33, 33, 33, 33]


100%|██████████| 33/33 [00:12<00:00,  2.62it/s]
OpenCV: FFMPEG: tag 0x5634504d/'MP4V' is not supported with codec id 12 and format 'mp4 / MP4 (MPEG-4 Part 14)'
OpenCV: FFMPEG: fallback to use tag 0x7634706d/'mp4v'


2 [33, 33, 33, 33]


...

100%|██████████| 33/33 [00:12<00:00,  2.62it/s]
OpenCV: FFMPEG: tag 0x5634504d/'MP4V' is not supported with codec id 12 and format 'mp4 / MP4 (MPEG-4 Part 14)'
OpenCV: FFMPEG: fallback to use tag 0x7634706d/'mp4v'


50 [33, 33, 33, 33]


100%|██████████| 33/33 [00:13<00:00,  2.51it/s]
OpenCV: FFMPEG: tag 0x5634504d/'MP4V' is not supported with codec id 12 and format 'mp4 / MP4 (MPEG-4 Part 14)'
OpenCV: FFMPEG: fallback to use tag 0x7634706d/'mp4v'


51 [33, 33, 33, 33]


100%|██████████| 33/33 [00:12<00:00,  2.63it/s]


In [26]:
# read from rectified data
id = 'unique_id'  # id or unique_id
for i,seq_df in enumerate(seq_dfs):
    # check model prediction and filter by dust/seg outputs
    pred_seq_df = pred_df[pred_df[id].isin(seq_df[id])]
    labeled_ids = pred_seq_df[id].to_list()

    frame = read_rectified_image(data_dir, seq_df.iloc[0])
    height, width, layers = frame.shape
    # print(height, width, layers)

    # .avi MJPG,  .mp4 MP4V
    video_dir = os.path.join(data_dir, f'videos')
    os.makedirs(video_dir, exist_ok=True)
    video_name = os.path.join(video_dir, f'seq{str(i).zfill(3)}_{seq_df.iloc[0].camera_pair}.mp4')
    video = cv2.VideoWriter(video_name, cv2.VideoWriter_fourcc(*'MP4V'), 6, (width,height), isColor=True)

    for ii in tqdm(range(len(seq_df))):
        try:
            cam_df_row = seq_df.iloc[ii]
            frame = read_rectified_image(data_dir, cam_df_row)
            c = (255,0,0)
            s = f'{cam_df_row.collected_on}'
            if cam_df_row[id] in labeled_ids:
                pred_row = pred_seq_df[pred_seq_df[id] == cam_df_row[id]].iloc[0]
                # s += f' pred: {pred_row.state}, dust: {pred_row.total_averaged_dust_conf:.4f}'
                s += f' pred: {pred_row.state}'
                if pred_row.state == 'true_positive' or pred_row.state == 'false_positive':
                    c = (0,0,255) 
            frame = cv2.putText(frame, s, 
                    (10,40), cv2.FONT_HERSHEY_SIMPLEX, 1, c, 1, cv2.LINE_AA)
            video.write(frame)
        except:
            print(f'{ii}th image read failed')

    cv2.destroyAllWindows()
    video.release()
    # break

OpenCV: FFMPEG: tag 0x5634504d/'MP4V' is not supported with codec id 12 and format 'mp4 / MP4 (MPEG-4 Part 14)'
OpenCV: FFMPEG: fallback to use tag 0x7634706d/'mp4v'
100%|██████████| 516/516 [00:26<00:00, 19.76it/s]
OpenCV: FFMPEG: tag 0x5634504d/'MP4V' is not supported with codec id 12 and format 'mp4 / MP4 (MPEG-4 Part 14)'
OpenCV: FFMPEG: fallback to use tag 0x7634706d/'mp4v'
100%|██████████| 520/520 [00:19<00:00, 26.03it/s]
OpenCV: FFMPEG: tag 0x5634504d/'MP4V' is not supported with codec id 12 and format 'mp4 / MP4 (MPEG-4 Part 14)'
OpenCV: FFMPEG: fallback to use tag 0x7634706d/'mp4v'
100%|██████████| 520/520 [00:29<00:00, 17.43it/s]
OpenCV: FFMPEG: tag 0x5634504d/'MP4V' is not supported with codec id 12 and format 'mp4 / MP4 (MPEG-4 Part 14)'
OpenCV: FFMPEG: fallback to use tag 0x7634706d/'mp4v'
100%|██████████| 505/505 [00:29<00:00, 17.41it/s]
OpenCV: FFMPEG: tag 0x5634504d/'MP4V' is not supported with codec id 12 and format 'mp4 / MP4 (MPEG-4 Part 14)'
OpenCV: FFMPEG: fallback

In [28]:
for i, seq_df in enumerate(seq_dfs):
    pred_seq_df = pred_df[pred_df[id].isin(seq_df[id])]
    print(i, seq_df.special_notes.unique(), pred_seq_df[pred_seq_df.state == 'false_positive'].shape)

0 ['tall stalks with blowing residue'] (1, 6)
1 ['tall stalks with blowing residue'] (0, 6)
2 ['tall stalks with blowing residue'] (0, 6)
3 ['tall stalks with blowing residue'] (0, 6)
4 ['tall stalks with blowing residue'] (0, 6)
5 ['tall stalks with blowing residue'] (0, 6)
6 ['tall stalks with blowing residue'] (1, 6)
7 ['tall stalks with blowing residue'] (0, 6)
8 ['tall stalks with blowing residue'] (0, 6)
9 ['tall stalks with blowing residue'] (0, 6)
10 ['tall stalks with blowing residue'] (0, 6)
11 ['tall stalks with blowing residue'] (0, 6)
12 ['tall stalks with blowing residue'] (44, 6)
13 ['tall stalks with blowing residue'] (15, 6)
14 ['tall stalks with blowing residue'] (6, 6)
15 ['tall stalks with blowing residue'] (55, 6)
16 ['tall stalks with blowing residue'] (52, 6)
17 ['tall stalks with blowing residue'] (15, 6)
18 ['tall stalks with blowing residue'] (1, 6)
19 ['tall stalks with blowing residue'] (0, 6)
20 ['tall stalks with blowing residue'] (56, 6)
21 ['tall stalks 

## Create video from saved pngs

In [10]:
def create_video(ids, pred_dir, video_name, read_func=read_saved_frame, fps=2, txt_df=None):
    frame = read_func(pred_dir, ids[0])
    height, width, layers = frame.shape
    print(height, width, layers)

    # .avi MJPG,  .mp4 MP4V
    video = cv2.VideoWriter(video_name, cv2.VideoWriter_fourcc(*'MP4V'), fps, (width,height), isColor=True)
    
    good = 0
    for _id in tqdm(ids):
        frame = read_func(pred_dir, _id)
        if frame is not None:
            if txt_df is not None:
                txt_row = txt_df[txt_df.unique_id == _id].iloc[0]
                c = (255,0,0)
                s = f'{txt_row.collected_on} pred state: {txt_row.result_state}'
                # s = f'{txt_row.collected_on} dust ratio: {txt_row.total_averaged_dust_conf:.4f}'
                frame = cv2.putText(frame, s, 
                        (600,60), cv2.FONT_HERSHEY_SIMPLEX, 1, c, 2, cv2.LINE_AA)
            video.write(frame)
            good += 1
    print('total', len(ids), 'used', good)

    # cv2.destroyAllWindows()
    video.release()

In [13]:
data_dir = '/data/jupiter/datasets/'
dataset = 'dust_datasets/halo_water_drops_on_lens_with_human_collection_0905'
master_df = pd.read_csv(os.path.join(data_dir, dataset, 'master_annotations.csv'))
master_df = master_df.sort_values('collected_on')

pred_root_dir = '/data/jupiter/li.yu/exps/driveable_terrain_model/'
model = '11_1_rev1_train_human_test_dean_multires'
pred_dir = os.path.join(pred_root_dir, model, dataset)
pred_df = pd.read_csv(os.path.join(pred_dir, 'output.csv'))
pred_df = master_df[['unique_id', 'collected_on']].merge(pred_df, on='unique_id')
# dust_df = pd.read_csv(os.path.join(pred_dir, 'dust_ratio.csv'))
# dust_df = master_df[['unique_id', 'collected_on']].merge(dust_df, on='unique_id')
save_dir = pred_dir

# # save as a single video
# print(master_df.shape)
# video_name = os.path.join(save_dir, 'pred.mp4')
# ids = master_df.image_id.to_list()
# create_video(ids, pred_dir, video_name, fps=3)

# break into sequences
seq_dfs = get_sequences(master_df, interval=60, per_camera=False, per_camera_pair=True)
print(len(master_df), len(seq_dfs))
video_dir = os.path.join(pred_dir, 'videos')
os.makedirs(video_dir, exist_ok=True)

for si, seq_df in enumerate(seq_dfs):
    if len(seq_df) < 10:
        continue
        
    name = seq_df.iloc[0].collected_on
    camera = seq_df.iloc[0].unique_id[-7:]
    print(si, name, camera, len(seq_df))
    
    # video_name = os.path.join(video_dir, str(si).zfill(3)+'.mp4')
    video_name = os.path.join(video_dir, f'{camera}_{si}.mp4')
    ids = seq_df.unique_id.to_list()
    create_video(ids, pred_dir, video_name, fps=3, txt_df=pred_df)

### Create video from PP artifacts

In [16]:
def read_from_pp_artifacts(data_dir, df_row):
    data_path = os.path.join(data_dir, df_row.stereo_pipeline_npz_save_path)
    img = np.load(data_path)['left']
    img_norm = normalize_image(img, df_row.hdr_mode if 'hdr_mode' in df_row else True)
    return cv2.cvtColor((img_norm * 255).astype(np.uint8), cv2.COLOR_RGB2BGR)

# def add_text(frame, txt_row):
#     frame = cv2.putText(frame, f'Collected on: {txt_row.collected_on}', 
#                         (40,50), cv2.FONT_HERSHEY_SIMPLEX, 1, (255,0,0), 2, cv2.LINE_AA)
#     return frame

def add_text(frame, txt):
    frame = cv2.putText(frame, txt, 
                        (40,50), cv2.FONT_HERSHEY_SIMPLEX, 1, (255,0,0), 2, cv2.LINE_AA)
    return frame

def add_texts(frame, txts: list):
    txt_pw, txt_ph = 10, 25
    for i, txt in enumerate(txts):
        if 'false_positive' in txt:
            frame = cv2.putText(frame, txt, 
                                (txt_pw, txt_ph+i*40), cv2.FONT_HERSHEY_SIMPLEX, 1, (0,0,255), 2, cv2.LINE_AA)
        else:
            frame = cv2.putText(frame, txt, 
                                (txt_pw, txt_ph+i*40), cv2.FONT_HERSHEY_SIMPLEX, 1, (255,0,0), 2, cv2.LINE_AA)
    return frame

In [5]:
# data_root_dir = '/data/jupiter/li.yu/data'
data_root_dir = '/data/jupiter/datasets/'
# dataset = 'mannequin_in_dust_v1'
# dataset = 'Jupiter_human_on_path_3_fn_sequence'
dataset = 'halo_missed_lo_rock_0509_stereo'
data_dir = os.path.join(data_root_dir, dataset)

df = pd.read_csv(os.path.join(data_dir, 'master_annotations.csv'))
seq_dfs = get_sequences(df, interval=5, per_camera=True)
print(df.shape, len(seq_dfs))

/home/li.yu/anaconda3/envs/pytorchlightning/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3437: DtypeWarning: Columns (37,46,58,60,74,86,101,103,106,116,118) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


(23023, 150) 432


In [6]:
bags = [
["05_09_2024_18_16_02", "05_09_2024_18_17_02"],
["05_09_2024_18_17_01", "05_09_2024_18_18_01"],
["05_09_2024_18_17_48", "05_09_2024_18_18_48"],
["05_09_2024_18_18_45", "05_09_2024_18_19_45"],
["05_09_2024_18_21_31", "05_09_2024_18_22_31"],
["05_09_2024_18_22_26", "05_09_2024_18_23_26"],
["05_09_2024_18_23_12", "05_09_2024_18_24_12"],
["05_09_2024_18_24_15", "05_09_2024_18_25_15"],
["05_09_2024_20_16_20", "05_09_2024_20_17_20"],
["05_09_2024_20_18_38", "05_09_2024_20_19_38"],
["05_09_2024_20_20_28", "05_09_2024_20_21_28"],
]
bags = [[bag[0][11:].replace('_', ':'), bag[1][11:].replace('_', ':')] for bag in bags]
df['collected_hms'] = df['collected_on'].apply(lambda t: t[11:])
df_oi = []
for bag in bags:
    sub_df = df[(df.collected_hms >= bag[0]) & (df.collected_hms < bag[1])]
    print(bag, len(sub_df))
    df_oi.append(sub_df)
df_oi = pd.concat(df_oi, ignore_index=True).drop_duplicates()
print(df_oi.shape)

seq_dfs = get_sequences(df_oi)
print(df_oi.shape, len(seq_dfs))

['18:16:02', '18:17:02'] 606
['18:17:01', '18:18:01'] 750
['18:17:48', '18:18:48'] 498
['18:18:45', '18:19:45'] 354
['18:21:31', '18:22:31'] 732
['18:22:26', '18:23:26'] 900
['18:23:12', '18:24:12'] 426
['18:24:15', '18:25:15'] 432
['20:16:20', '20:17:20'] 930
['20:18:38', '20:19:38'] 850
['20:20:28', '20:21:28'] 1200
(6568, 151)
(6568, 151) 2


In [8]:
pred_dir = '/data/jupiter/li.yu/exps/driveable_terrain_model'
models = [
    'ds_v8_1_nextvit_small_openimages_with_rev1_train_human_test_using_random_val_mhc_20_epochs_finetune_rev1_lr',
    'v81_80k_maxfov_wn_ft_kore_0430'
]
suffix_list = ['_mhc_depth0125', '']
pred_dfs = [pd.read_csv(os.path.join(pred_dir, model, dataset+suffix, 'output.csv')) for model,suffix in zip(models, suffix_list)]
pred_dfs[0].head(1)

,unique_id,id,camera_location,operation_time,special_notes,jdb_s3_path,result_state,result_human_state,result_vehicle_state,min_pixels_threshold,features,n_gt_human_pixels,gt_human_depth,n_pred_human_pixels,pred_human_depth
0,663ec898321f043ed7ad8a62_T02_T03,663ec898321f043ed7ad8a62,T02,daytime,NaN,NaN,true_negative,true_negative,true_negative,108,"{""large_object_pixels"": 688, ""large_object_min...",0,1000,0,1000


In [18]:
def get_pred_texts(pred_dfs, unique_id, text_prefix):
    texts = []
    for pred_df, prefix in zip(pred_dfs, text_prefix):
        pred_row = pred_df[pred_df.unique_id == unique_id].iloc[0]
        texts.append(f'{prefix}: {pred_row.result_state}')
    return texts

def create_video_from_pp_add_text(seq_df, camera_pairs, pred_dfs, H, W, data_dir, video_dir_name):
    cam_dfs = [seq_df[seq_df.unique_id.str.endswith(c)] for c in camera_pairs]
    cam_dfs = [cdf.sort_values('collected_on', ignore_index=True) for cdf in cam_dfs]
    min_len = min(len(cdf) for cdf in cam_dfs)
    cam_dfs = [cdf.iloc[:min_len] for cdf in cam_dfs]
    print([len(cdf) for cdf in cam_dfs])

    # .avi MJPG,  .mp4 MP4V
    os.makedirs(os.path.join(data_dir, video_dir_name), exist_ok=True)
    video_name = os.path.join(data_dir, f'{video_dir_name}/{seq_df.iloc[0].collected_on}.mp4')

    # print(f'{H} rows, {W} cols out of {len(camera_pairs)} camera_pairs')
    video = cv2.VideoWriter(video_name, cv2.VideoWriter_fourcc(*'MP4V'), 3, (768*W,512*H), isColor=True)

    for ii in tqdm(range(min_len)):
        # try:
        canvas = np.zeros((512*H, 768*W, 3), dtype=np.uint8)
        for ci in range(len(cam_dfs)):
            row = cam_dfs[ci].iloc[ii]
            frame = read_from_pp_artifacts(data_dir, row)
            texts = [f'{row.collected_on}'] + get_pred_texts(pred_dfs, row.unique_id, ['MHC pred', 'MAXFOV pred'])
            # print(texts)
            frame = add_texts(frame, texts)
            fi, fj = ci // W, ci % W
            canvas[512*fi:512*(fi+1), 768*fj:768*fj+frame.shape[1]] = frame
        video.write(canvas)
        # except:
        #     print(f'{ii}th image read failed')

    # cv2.destroyAllWindows()
    video.release()

front_pairs = ['T02_T04', 'T02_T03', 'T01_T03']
left_pass_pairs = ['T02_T04', 'T02_T03', 'T01_T03', 'T13_T15', 'T14_T15', 'T14_T16']
right_pass_pairs = ['T02_T04', 'T02_T03', 'T01_T03', 'T06_T08', 'T05_T07', 'T06_T07']
# left_pass_pairs = ['T09_T11', 'T14_T16', 'T14_T15', 'T13_T15']
# right_pass_pairs = ['T05_T07', 'T10_T12', 'T06_T08', 'T06_T07']
H = 1  # number of image rows
W = 3  # number of image cols
create_video_from_pp_add_text(seq_dfs[1], front_pairs, pred_dfs, H, W, data_dir, 'videos_front_pass')

  0%|          | 1/248 [00:00<00:29,  8.30it/s]

[248, 248, 248]


100%|██████████| 248/248 [00:30<00:00,  8.21it/s]


In [36]:
df["camera_pair"] = df["unique_id"].apply(lambda t: t[-7:])
df["camera_pair"].unique()

array(['T01_T03', 'T02_T03', 'T02_T04', 'T05_T07', 'T06_T07', 'T06_T08',
       'T09_T11', 'T10_T12', 'T10_T11', 'T13_T15', 'T14_T16', 'T14_T15'],
      dtype=object)

### Read frame and add prediction results

In [20]:
# compare BRT model pred and CenterTrack pred
# pred_csv = '/data/jupiter/li.yu/exps/driveable_terrain_model/v188_58d_rak_local_fine_tversky11_sum_image_normT_prod5_airdyn_r3a8_s30/mannequin_in_dust_v1/output.csv'
pred_csv = '/data/jupiter/li.yu/exps/driveable_terrain_model/v188_58d_rak_local_fine_tversky11_sum_image_normT_prod5_airdyn_r3a8_s30/Jupiter_human_on_path_3_fn_sequence/output.csv'
pred_df = pd.read_csv(pred_csv)
print(pred_df.shape)

(208, 11)


In [11]:
video_with_pred_dir = os.path.join(data_root_dir, dataset, 'videos_with_pred')
# pred_sequence_dir = '/home/li.yu/code/CenterTrack/results/2023-07-08T01:37:09.798000_front-center_15'
pred_sequence_dir = '/home/li.yu/code/CenterTrack/results/brt50000/nopreimg_noprehm/2023-07-08T01:37:09.798000_front-center_15'
os.makedirs(video_with_pred_dir, exist_ok=True)
height, width = 512, 1024

for si, seq_df in enumerate(seq_dfs):
    if len(seq_df) < 10 or si != 15:
        continue
    name = seq_df.iloc[0].collected_on
    camera = seq_df.iloc[0].camera_location[:-5]
    print(si, name, len(seq_df))
    
    # merge pred from BRT model
    seq_df = seq_df.drop(columns=['state']).merge(pred_df[['id', 'state', 'human_state']], on='id')

    # create video
    video_name = os.path.join(video_with_pred_dir, f'{name}_{camera}_{si}_nopreimg_noprehm.mp4')
    video = cv2.VideoWriter(video_name, cv2.VideoWriter_fourcc(*'mp4v'), 3, (width,height), isColor=True)
    fi = 0
    for _, df_row in seq_df.iterrows():
        frame = cv2.imread(os.path.join(pred_sequence_dir, str(fi).zfill(3)+'_'+df_row.id+'.png'))
        frame = cv2.putText(frame, f'BRT Seg Pred: {df_row.state}, Strict: {df_row.human_state}', 
                            (40,50), cv2.FONT_HERSHEY_SIMPLEX, 1, (255,0,0), 2, cv2.LINE_AA)
        video.write(frame)
        fi += 1
    cv2.destroyAllWindows()
    video.release()

    # break


15 2023-07-08T01:37:09.798000 153


### read from video and prediction results to each frame

In [23]:
video_dir = '/home/li.yu/code/CenterTrack/results/'
old_video_name = 'brt50000_2022-11-10T23:19:10.901000_side-right_2_preimg.mp4'
new_video_name = 'brt50000_2022-11-10T23:19:10.901000_side-right_2_preimg_withbrtpred.mp4'
height, width = 512, 1024

for si, seq_df in enumerate(seq_dfs):
    if len(seq_df) < 10 or si != 2:
        continue
    name = seq_df.iloc[0].collected_on
    camera = seq_df.iloc[0].camera_location[:-5]
    print(si, name, len(seq_df))
    
    # merge pred from BRT model
    seq_df = seq_df.drop(columns=['state']).merge(pred_df[['id', 'state', 'human_state']], on='id')

    # read video
    cam = cv2.VideoCapture(os.path.join(video_dir, old_video_name))

    # create video
    video_name = os.path.join(video_dir, new_video_name)
    video = cv2.VideoWriter(video_name, cv2.VideoWriter_fourcc(*'mp4v'), 3, (width,height), isColor=True)
    for _, df_row in seq_df.iterrows():
        _, frame = cam.read()
        frame = cv2.putText(frame, f'BRT Seg Pred: {df_row.state}, Strict: {df_row.human_state}', 
                            (40,50), cv2.FONT_HERSHEY_SIMPLEX, 1, (255,0,0), 2, cv2.LINE_AA)
        video.write(frame)
    cv2.destroyAllWindows()
    video.release()

    # break


2 2022-11-10T23:19:10.901000 88
